### Style Backend

In [27]:
# ==== Cell 0 — Whiteboard UI Kit (run once) ====
from IPython.display import HTML, display

# Respect existing globals, provide safe defaults
ACCENT      = globals().get("ACCENT",      "#74C69D")
HEADING     = globals().get("HEADING",     "#0F172A")
CARD_BG     = globals().get("CARD_BG",     "#FFFFFF")
CARD_BORDER = globals().get("CARD_BORDER", "rgba(15, 23, 42, 0.08)")
INFO_BG     = globals().get("INFO_BG",     "#F0FAF5")
INFO_BORDER = globals().get("INFO_BORDER", "#D7F2E2")

# Provide THEME_A / THEME_B fallbacks if not already defined
if "THEME_A" not in globals() or not isinstance(globals().get("THEME_A"), dict):
    THEME_A = {
        "accent": ACCENT,
        "heading": HEADING,
        "card_bg": CARD_BG,
        "card_border": CARD_BORDER,
        "bar": "#2CA58D",
    }
    globals()["THEME_A"] = THEME_A

if "THEME_B" not in globals() or not isinstance(globals().get("THEME_B"), dict):
    THEME_B = {
        "accent": "#60A5FA",
        "heading": "#0F172A",
        "card_bg": "#FFFFFF",
        "card_border": "rgba(15, 23, 42, 0.08)",
        "bar": "#3B82F6",
    }
    globals()["THEME_B"] = THEME_B

# Whiteboard shadow stack
WB_SHADOW = ("0 1px 2px rgba(15, 23, 42, 0.06), "
             "0 8px 32px rgba(15, 23, 42, 0.08)")

# Optional faint paper gradient
WB_BG_GRADIENT = "linear-gradient(180deg, #FFFFFF 0%, #FBFBFD 100%)"

display(HTML(f"""
<style>
:root {{
  --wb-accent: {ACCENT};
  --wb-heading:{HEADING};
  --wb-card-bg:{CARD_BG};
  --wb-card-bdr:{CARD_BORDER};
  --wb-info-bg:{INFO_BG};
  --wb-info-bdr:{INFO_BORDER};
}}

.wb-card {{
  background: var(--wb-card-bg);
  border: 1px solid var(--wb-card-bdr);
  border-radius: 16px;
  box-shadow: {WB_SHADOW};
}}

.wb-paper {{
  background: {WB_BG_GRADIENT};
  border: 1px solid var(--wb-card-bdr);
  border-radius: 18px;
  box-shadow: {WB_SHADOW};
}}

.wb-title {{
  font-family: ui-sans-serif, system-ui, -apple-system, Segoe UI, Roboto, Helvetica, Arial;
  color: var(--wb-heading);
  font-weight: 800; letter-spacing: .2px;
}}

.wb-subtle {{ color:#64748b; }}
.wb-chip {{
  display:inline-flex; align-items:center; gap:8px;
  padding:4px 10px; border-radius:999px; font-weight:700; font-size:12px;
  background:#fff; border:1px solid var(--wb-card-bdr); color: var(--wb-heading);
  box-shadow: 0 1px 2px rgba(0,0,0,.04);
}}

/* ipywidgets polish (shared) */
.wb-skin .widget-button {{
  border-radius: 12px !important; border: 0 !important;
  background: var(--wb-accent) !important; color: #fff !important;
  box-shadow: 0 6px 18px rgba(116,198,157,.35); font-weight: 700;
}}
.wb-skin .widget-button:focus {{ outline: 2px solid #a3e7c7; }}

.wb-skin .widget-toggle-buttons .widget-toggle-button {{
  border:1px solid var(--wb-card-bdr);
  background:#fff; color:#0f172a; border-radius:999px !important; margin:2px;
  box-shadow: 0 1px 2px rgba(16,24,40,.06) inset;
}}
.wb-skin .widget-toggle-buttons .widget-toggle-button.mod-active,
.wb-skin .widget-toggle-buttons .widget-toggle-button[aria-pressed="true"],
.wb-skin .widget-toggle-buttons .widget-toggle-button[aria-checked="true"] {{
  background: var(--wb-accent) !important; color:#fff !important; border-color: var(--wb-accent) !important;
  box-shadow: 0 0 0 2px rgba(116,198,157,.25) inset;
}}
.wb-skin .widget-checkbox input {{ accent-color: var(--wb-accent) !important; }}

/* Transparent pandas Styler tables globally */
table.dataframe, table.dataframe thead tr, table.dataframe tbody tr, 
table.dataframe th, table.dataframe td {{ background: transparent !important; }}
</style>
"""))


## Viability Assessment

### Backend

In [19]:
# ==== Cell 1 — Viability Assessment (whiteboard-styled) ====
import re, traceback, numpy as np, pandas as pd, ipywidgets as w
from pathlib import Path
from IPython.display import display, HTML, clear_output

# ===================== 1) Load & normalize rules =====================
LAST_RULES_SOURCE = None
LAST_RULES_SHEET  = None

def _canon(s: str) -> str:
    s = str(s).strip().lower()
    s = re.sub(r"[\s\-]+", "_", s)
    s = s.replace("≤","le").replace("≥","ge").replace("<=","le").replace(">=","ge")
    s = s.replace("<","lt").replace(">","gt").replace(".","_").replace("/","_")
    s = re.sub(r"[^a-z0-9_]+","", s)
    return s

EXPECTED_ORDER = [
    "MMC Method",
    "Low Rise","Medium Rise","High Rise",
    "On Site Storage","Off Site Storage",
    "Tower Crane","Telescopic Crane","Telehandler Crane",
    "Flatbed Truck","Flatbed A Frame",
    "Max Panel Height (m)","Max Frame Length (m)",
    "Max Frame Width Less Than 3,2 m","Max Frame Width More Than 3,2 m",
    "Small Units","Geometry Bypass"
]
EXPECTED_CANON_MAP = {_canon(c): c for c in EXPECTED_ORDER}
BOOL_COLS = [c for c in EXPECTED_ORDER if c not in ["MMC Method","Max Panel Height (m)","Max Frame Length (m)"]]
NUM_COLS  = ["Max Panel Height (m)","Max Frame Length (m)"]

def _to_bool(v):
    if pd.isna(v): return False
    if isinstance(v, (bool, np.bool_)): return bool(v)
    if isinstance(v, (int, float, np.integer, np.floating)): return bool(int(v))
    s = str(v).strip().lower()
    truthy = {"true","t","yes","y","1","ok","✓","x","evet"}
    falsey = {"false","f","no","n","0","none","na","n/a","-","hayir","hayır"}
    if s in truthy: return True
    if s in falsey: return False
    return False

def _to_float(v):
    if pd.isna(v): return np.nan
    if isinstance(v, (int, float, np.integer, np.floating)): return float(v)
    s = str(v).strip()
    if s in {"", "-", "na", "n/a", "none"}: return np.nan
    s = s.replace(",", ".")
    m = re.search(r"([-+]?\d+(?:\.\d+)?)", s)
    return float(m.group(1)) if m else np.nan

def _pick_viab_sheet(xls: pd.ExcelFile) -> str:
    for s in xls.sheet_names:
        if "viab" in _canon(s):
            return s
    return xls.sheet_names[0]

def _load_rules_frame() -> pd.DataFrame:
    global LAST_RULES_SOURCE, LAST_RULES_SHEET
    candidates = [
        Path("mmc_viability_data.xlsx"),
        Path("/mnt/data/mmc_viability_data.xlsx"),
        Path("mmc_viability_updated.xlsx"),
        Path("/mnt/data/mmc_viability_updated.xlsx"),
    ]
    err = []
    for p in candidates:
        try:
            if not p.exists():
                err.append(f"not found: {p}")
                continue
            xls = pd.ExcelFile(p)
            sheet = _pick_viab_sheet(xls)
            df = pd.read_excel(p, sheet_name=sheet)
            LAST_RULES_SOURCE, LAST_RULES_SHEET = str(p), sheet
            return df
        except Exception as e:
            err.append(f"{p}: {e}")
            continue
    raise RuntimeError("Cannot load viability rules from any candidate file.\n" + "\n".join(err))

def normalize_rules_df(df: pd.DataFrame) -> pd.DataFrame:
    rename_map = {}
    for col in df.columns:
        c = _canon(col)
        if c in EXPECTED_CANON_MAP:
            rename_map[col] = EXPECTED_CANON_MAP[c]
    df = df.rename(columns=rename_map)

    if "MMC Method" not in df.columns:
        alts = [c for c in df.columns if _canon(c) in {"mmc_method","mmc","mmcmethod","method","system_type","system"}]
        if alts:
            df["MMC Method"] = df[alts[0]].astype(str)
        else:
            maybes = [c for c in df.columns if "method" in _canon(c) or "mmc" in _canon(c)]
            if maybes:
                df["MMC Method"] = df[maybes[0]].astype(str)
            else:
                raise RuntimeError("Rules table has no MMC method column. Add a column named 'MMC Method'.")

    for c in EXPECTED_ORDER:
        if c not in df.columns:
            df[c] = pd.NA

    for c in BOOL_COLS:
        if c in df.columns: df[c] = df[c].map(_to_bool)
    for c in NUM_COLS:
        if c in df.columns: df[c] = df[c].map(_to_float)

    df["MMC Method"] = df["MMC Method"].astype(str).str.strip()
    df = df[df["MMC Method"].str.len().gt(0)].reset_index(drop=True)

    expected_first = [c for c in EXPECTED_ORDER if c in df.columns]
    the_rest = [c for c in df.columns if c not in expected_first]
    return df[expected_first + the_rest]

# Load once
try:
    rules  # reuse if present
except NameError:
    rules = None
if not isinstance(rules, pd.DataFrame) or rules.empty:
    raw_rules = _load_rules_frame()
    rules = normalize_rules_df(raw_rules)

# ===================== 2) Evaluator =====================
def _evaluate_viability_row(row: pd.Series, answers: dict, strict_unknown=True):
    reasons, notes = [], []
    # Rise band
    band_map = {"low":"Low Rise","med":"Medium Rise","high":"High Rise"}
    rise_col = band_map[answers["band"]]
    if not bool(row.get(rise_col, False)):
        pretty = {"low":"Low rise (1–3)","med":"Medium rise (4–9)","high":"High rise (10+)"}[answers["band"]]
        reasons.append(f"Not suitable for {pretty} buildings.")
    # Storage
    loc_col = "On Site Storage" if answers["storage_type"] == "on" else "Off Site Storage"
    if not bool(row.get(loc_col, False)):
        pretty = "On-site" if answers["storage_type"] == "on" else "Off-site"
        reasons.append(f"{pretty} storage not supported.")
    # Cranes
    site_cranes = {"Tower Crane": answers["tower"], "Telescopic Crane": answers["telesc"], "Telehandler Crane": answers["teleh"]}
    site_has_crane = any(site_cranes.values())
    method_cranes = [k for k in site_cranes if bool(row.get(k, False))]
    if site_has_crane:
        overlap = [k for k in site_cranes if site_cranes[k] and bool(row.get(k, False))]
        if not overlap: reasons.append("No compatible crane with site availability.")
    else:
        if len(method_cranes) > 0:
            pretty = ", ".join(t.replace("Crane","crane") for t in method_cranes)
            reasons.append(f"Method requires a crane ({pretty}), but none are available on site.")
    # Trucks
    site_trucks = {"Flatbed Truck": answers["flatbed"], "Flatbed A Frame": answers["aframe"]}
    site_has_truck = any(site_trucks.values())
    method_trucks = [k for k in site_trucks if bool(row.get(k, False))]
    if site_has_truck:
        overlap = [k for k in site_trucks if site_trucks[k] and bool(row.get(k, False))]
        if not overlap: reasons.append("No compatible delivery truck with site availability.")
    else:
        if len(method_trucks) > 0:
            pretty = ", ".join(t.replace("Flatbed","flatbed") for t in method_trucks)
            reasons.append(f"Method requires a delivery truck ({pretty}), but none are available on site.")
    # Geometry (bypass)
    mph = row.get("Max Panel Height (m)", np.nan)
    mfl = row.get("Max Frame Length (m)", np.nan)
    bypass = bool(row.get("Small Units", False) or row.get("Geometry Bypass", False))
    if not bypass:
        bypass = ((pd.notna(mph) and float(mph) <= 0.5) and (pd.notna(mfl) and float(mfl) <= 1.0))
    if bypass:
        notes.append("Geometry (panel height & frame length) ignored due to small discrete units.")
    else:
        req_ph = float(answers["panel_h_need"])
        if pd.isna(mph):
            if strict_unknown: reasons.append("Panel height limit is unspecified in rules.")
        else:
            if req_ph > float(mph): reasons.append(f"Panel height {req_ph:.2f} m exceeds limit {float(mph):.2f} m.")
        op, val = answers["frame_len_need"]
        if pd.isna(mfl):
            if strict_unknown: reasons.append("Frame length limit is unspecified in rules.")
        else:
            mfl = float(mfl)
            if op == "le" and float(val) > mfl: reasons.append(f"Frame length requirement ≤ {val:.2f} m exceeds method limit {mfl:.2f} m.")
            if op == "gt" and mfl <= float(val): reasons.append(f"Frame length requirement > {val:.2f} m not supported (limit {mfl:.2f} m).")
    # Frame width band
    if answers["width_over"]:
        if not bool(row.get("Max Frame Width More Than 3,2 m", False)):
            reasons.append("Frame width > 3.2 m not supported.")
    else:
        if not bool(row.get("Max Frame Width Less Than 3,2 m", False)):
            reasons.append("Frame width ≤ 3.2 m not supported.")
    return (len(reasons) == 0), reasons, notes

def evaluate_all(rules_df: pd.DataFrame, answers: dict, strict_unknown=True):
    rows = []
    for _, r in rules_df.iterrows():
        ok, reasons, notes = _evaluate_viability_row(r, answers, strict_unknown=strict_unknown)
        msg = "; ".join(reasons)
        if notes:
            msg = (msg + ("; " if msg else "")) + " | ".join([f"Note: {n}" for n in notes])
        rows.append({"MMC Method": r["MMC Method"], "Viable": bool(ok), "Reasons": msg})
    return pd.DataFrame(rows)

# ===================== 3) UI + rendering =====================
def render_results_nice(df: pd.DataFrame) -> str:
    ok = df[df["Viable"]==True]["MMC Method"].tolist()
    ko = df[df["Viable"]==False][["MMC Method","Reasons"]].values.tolist()
    n_all = int(len(df))
    n_ok  = int(len(ok))
    pct   = (100.0 * n_ok / n_all) if n_all else 0.0

    css = f"""
    <style>
      :root {{
        --acc: {ACCENT};
        --heading: {HEADING};
        --card-bg: {CARD_BG};
        --card-bdr: {CARD_BORDER};
        --info-bg: {INFO_BG};
        --info-bdr: {INFO_BORDER};
        --ink: #0f172a;
        --muted: #64748b;
        --ok: #137333;
        --ko: #b42318;
      }}
      .mmc3-wrap {{ font-family: ui-sans-serif, system-ui, -apple-system, Segoe UI, Roboto, Helvetica, Arial; color: var(--ink); }}
      .mmc3-grid {{ display: grid; grid-template-columns: 1fr; gap: 14px; }}
      @media (min-width: 980px) {{ .mmc3-grid {{ grid-template-columns: 1fr 1fr; }} }}
      .mmc3-card {{
        background: var(--card-bg);
        border: 1px solid var(--card-bdr);
        border-radius: 16px;
        padding: 16px 18px;
        box-shadow: 0 1px 2px rgba(15,23,42,.06), 0 8px 32px rgba(15,23,42,.08);
      }}
      .mmc3-title {{ margin:0 0 10px 0; font-size:16px; font-weight:800; color:var(--heading); letter-spacing:.2px; display:flex; align-items:center; gap:8px; }}
      .mmc3-badges {{ display:flex; flex-wrap:wrap; gap:8px; }}
      .mmc3-badge-ok {{ padding:6px 10px; border-radius:999px; background:#e7f6ee; border:1px solid #c8eedc; color:var(--ok); font-weight:700; font-size:12px; }}
      .mmc3-chip {{ display:inline-flex; align-items:center; gap:6px; font-weight:700; font-size:12px; padding:4px 10px; border-radius:999px; background:var(--info-bg); border:1px solid var(--info-bdr); color:#1d3557; }}
      .mmc3-pill {{ display:inline-flex; align-items:center; gap:6px; padding:4px 8px; border-radius:999px; font-size:12px; color:var(--muted); background:#f6f7f8; border:1px solid #eceff1; }}
      .mmc3-progress {{ height:8px; border-radius:999px; background:#eef3ef; border:1px solid #dfe8e1; overflow:hidden; margin:6px 0 2px 0; }}
      .mmc3-progress > span {{ display:block; height:100%; width:{pct:.1f}%; background: linear-gradient(90deg, var(--acc), #6ed4a8); }}
      details.mmc3-item {{ border:1px dashed #e8ecea; border-radius:12px; padding:8px 10px; background:#fff; margin-bottom:8px; }}
      details.mmc3-item[open] {{ background:#fcfffd; }}
      details.mmc3-item summary {{ cursor:pointer; list-style:none; display:flex; align-items:center; gap:8px; }}
      details.mmc3-item summary::-webkit-details-marker {{ display:none; }}
      .mmc3-ko {{ padding:6px 10px; border-radius:999px; background:#fde8e8; color:var(--ko); font-weight:700; border:1px solid #f5caca; }}
      ul.mmc3-reasons {{ margin:8px 0 0 18px; }}
      .mmc3-empty {{ color:var(--muted); font-style:italic; }}
    </style>
    """
    parts = [css, '<div class="mmc3-wrap">']
    parts.append(f"""
      <div class="mmc3-chip" title="Shareable summary"><span>Viability</span><strong>{n_ok} / {n_all}</strong></div>
      <div class="mmc3-progress"><span></span></div>
      <div style="font-size:11px;color:var(--muted);margin-bottom:6px">{pct:.1f}% of methods meet current requirements.</div>
    """)
    parts.append('<div class="mmc3-grid">')
    parts.append('<div class="mmc3-card">')
    parts.append('<div class="mmc3-title">🌿 Viable methods</div>')
    if ok:
        parts.append('<div class="mmc3-badges">'); parts.extend([f'<span class="mmc3-badge-ok">{m}</span>' for m in ok]); parts.append('</div>')
    else:
        parts.append('<div class="mmc3-empty">No methods meet all requirements.</div>')
    parts.append('</div>')
    parts.append('<div class="mmc3-card">')
    parts.append('<div class="mmc3-title">⚠️ Not viable & reasons</div>')
    if ko:
        for m, rs in ko:
            items = [x.strip() for x in str(rs).split(";") if x and x.strip()]
            reasons = [x for x in items if not x.lower().startswith("note:")]
            notes   = [x for x in items if x.lower().startswith("note:")]
            parts.append('<details class="mmc3-item">')
            parts.append(f'<summary><span class="mmc3-ko">{m}</span><span class="mmc3-pill">{len(reasons)} reason(s)</span></summary>')
            if reasons: parts.append('<ul class="mmc3-reasons">' + "".join([f"<li>{r}</li>" for r in reasons]) + '</ul>')
            if notes:   parts.append('<div style="margin-top:6px;color:#566;font-style:italic;">' + " ".join(notes) + '</div>')
            if not reasons and not notes: parts.append('<div class="mmc3-empty">No specific reasons recorded.</div>')
            parts.append('</details>')
    else:
        parts.append('<div class="mmc3-empty">All methods are viable for the current inputs.</div>')
    parts.append('</div></div></div>')
    return "".join(parts)

def render_error(e: Exception) -> str:
    tb = traceback.format_exc(limit=3)
    src = f"{LAST_RULES_SOURCE or '—'} — sheet: {LAST_RULES_SHEET or '—'}"
    cols = ", ".join(list(rules.columns)) if isinstance(rules, pd.DataFrame) else "—"
    return f"""
    <div style="border:1px solid #F8D7DA;background:#FFF5F5;border-radius:12px;padding:12px">
      <div style="font-weight:700;color:#842029">Assessment error</div>
      <div style="color:#842029;margin-top:4px">{str(e)}</div>
      <div style="margin-top:6px;color:#6c757d"><b>Rules loaded from:</b> {src}</div>
      <div style="margin-top:4px;color:#6c757d"><b>Rules columns:</b> {cols}</div>
      <pre style="margin-top:8px;font-size:12px;white-space:pre-wrap">{tb}</pre>
    </div>
    """

# --------- Widgets ---------
res_band = w.ToggleButtons(options=[("Low (1–3)","low"), ("Medium (4–9)","med"), ("High (10+)","high")], value="low")
storage_type = w.ToggleButtons(options=[("On-site","on"), ("Off-site","off")], value="on")
tower   = w.Checkbox(description="Tower crane available", value=True)
telesc  = w.Checkbox(description="Telescopic crane available", value=False)
teleh   = w.Checkbox(description="Telehandler available", value=True)
flatbed = w.Checkbox(description="Flatbed truck available", value=True)
aframe  = w.Checkbox(description="Flatbed with A-frame", value=False)
panel_h_need = w.ToggleButtons(options=[("≤ 3.0 m", 3.0), ("> 3.0 m", 3.01)], value=3.0)
frame_len_need = w.ToggleButtons(options=[("≤ 12 m",("le", 12.0)), ("> 12 m",("gt", 12.0))], value=("le", 12.0))
width_over = w.ToggleButtons(options=[("Yes", True), ("No", False)], value=False)

apply_btn  = w.Button(description="Apply Viability", button_style="success", icon="check")
export_btn = w.Button(description="Export Results to Excel", button_style="info", icon="download")
reset_btn  = w.Button(description="Reset", icon="undo")
out = w.Output()
status_html = w.HTML(
    f"<div class='mmc3-subtle'>Loaded rules from <b>{LAST_RULES_SOURCE or '—'}</b> (sheet: <b>{LAST_RULES_SHEET or '—'}</b>) — {len(rules)} rows</div>"
)

def _gather_answers():
    return dict(
        band=res_band.value,
        storage_type=storage_type.value,
        tower=tower.value, telesc=telesc.value, teleh=teleh.value,
        flatbed=flatbed.value, aframe=aframe.value,
        panel_h_need=panel_h_need.value,
        frame_len_need=frame_len_need.value,
        width_over=width_over.value
    )

_latest_results = None
VIABLE_METHODS_SET = None

def _get_viable_methods_set_from_results(res_df: pd.DataFrame):
    if res_df is None or res_df.empty: return None
    ok = (res_df.loc[res_df['Viable'] == True, 'MMC Method']
          .dropna().astype(str).str.strip().str.casefold().tolist())
    return set(ok) if ok else None

def _push_viability_to_selection_ui():
    global VIABLE_METHODS_SET
    VIABLE_METHODS_SET = _get_viable_methods_set_from_results(_latest_results)
    if 'app' in globals() and hasattr(app, 'set_viable_methods'):
        app.set_viable_methods(VIABLE_METHODS_SET)

def on_apply(_=None):
    global _latest_results
    with out:
        clear_output()
        try:
            answers = _gather_answers()
            _latest_results = evaluate_all(rules, answers, strict_unknown=False)
            display(HTML(render_results_nice(_latest_results)))
            _push_viability_to_selection_ui()
            if 'refresh_mfg_map_from_viability' in globals():
                refresh_mfg_map_from_viability()
        except Exception as e:
            display(HTML(render_error(e)))

def on_export(_=None):
    with out:
        clear_output(wait=True)
        try:
            if _latest_results is None or _latest_results.empty:
                display(HTML("<div style='color:#8a1f1f'>No results yet. Click <b>Apply Viability</b> first.</div>"))
                return
            path = Path("viability_results.xlsx").absolute()
            try:
                writer = pd.ExcelWriter(path, engine="xlsxwriter")
            except Exception:
                writer = pd.ExcelWriter(path)
            with writer as wri:
                rules.to_excel(wri, index=False, sheet_name="rules_clean")
                pd.DataFrame([_gather_answers()]).to_excel(wri, index=False, sheet_name="inputs")
                _latest_results.to_excel(wri, index=False, sheet_name="results")
            display(HTML(render_results_nice(_latest_results)))
            display(HTML(f"<div style='margin-top:8px;color:#137333'>Exported to: <code>{path}</code></div>"))
        except Exception as e:
            display(HTML(render_error(e)))

def _reset(_=None):
    res_band.value = "low"
    storage_type.value = "on"
    tower.value, telesc.value, teleh.value = True, False, True
    flatbed.value, aframe.value = True, False
    panel_h_need.value = 3.0
    frame_len_need.value = ("le", 12.0)
    width_over.value = False
    with out: clear_output()
reset_btn.on_click(_reset)

apply_btn.on_click(on_apply)
export_btn.on_click(on_export)

# --------- Global style (whiteboard; incl. persistent selection highlight) ---------
display(HTML(f"""
<style>
  :root {{
    --acc: {ACCENT};
    --heading: {HEADING};
    --card-bg: {CARD_BG};
    --card-bdr: {CARD_BORDER};
    --info-bg: {INFO_BG};
    --info-bdr: {INFO_BORDER};
    --ink:#0f172a; --muted:#64748b;
  }}
  .mmc3-app {{
    background: var(--card-bg);
    border: 1px solid var(--card-bdr);
    border-radius: 18px;
    padding: 12px 14px;
    box-shadow: 0 1px 2px rgba(15,23,42,.06), 0 8px 32px rgba(15,23,42,.08);
    overflow-x: hidden;
  }}
  .mmc3-toolbar {{ display:flex; align-items:center; gap:10px; padding-bottom:6px; }}
  .mmc3-title {{ font-weight:800; color:var(--heading); font-size:16px; letter-spacing:.2px; }}
  .mmc3-subtle {{ color:var(--muted); font-size:12px; }}
  .mmc3-hr {{ border:0; border-top:1px dashed var(--card-bdr); margin:10px 0; }}
  .mmc3-label {{ font-weight:700; font-size:12px; color:#1f2937; margin:2px 0 6px; display:flex; align-items:center; gap:6px; }}
  .mmc3-app .widget-checkbox input {{ accent-color: var(--acc) !important; }}
  .mmc3-app .widget-toggle-buttons .widget-toggle-button {{
    border: 1px solid var(--card-bdr);
    background: #ffffff; color:#0f172a; border-radius: 999px !important; margin: 2px;
    box-shadow: 0 1px 2px rgba(16,24,40,.06) inset;
  }}
  .mmc3-app .widget-toggle-buttons .widget-toggle-button.mod-active,
  .mmc3-app .widget-toggle-buttons .widget-toggle-button[aria-pressed="true"],
  .mmc3-app .widget-toggle-buttons .widget-toggle-button[aria-checked="true"] {{
    background: var(--acc) !important; color: #ffffff !important; border-color: var(--acc) !important;
    box-shadow: 0 0 0 2px rgba(116,198,157,.25) inset;
  }}
  .mmc3-app .widget-button {{
    border-radius: 12px !important; border: 0 !important;
    background: var(--acc) !important; color: #fff !important;
    box-shadow: 0 6px 18px rgba(116,198,157,.35); font-weight: 700;
  }}
  .mmc3-app .widget-button:focus {{ outline: 2px solid #a3e7c7; }}
  .mmc3-app .widget-gridbox {{ overflow-x: hidden !important; }}
  .mmc3-app .widget-checkbox .widget-label {{ white-space: normal !important; }}
</style>
"""))

# Tooltips + compact heights
apply_btn.tooltip  = "Run the assessment"
export_btn.tooltip = "Save rules, inputs, and results to Excel"
reset_btn.tooltip  = "Reset all inputs"
for b in (apply_btn, export_btn, reset_btn):
    b.layout = w.Layout(height="36px")

# ------- Layout -------
row_primary = w.GridBox(
    children=[
        w.VBox([w.HTML("<div class='mmc3-label'>Residential type</div>"), res_band]),
        w.VBox([w.HTML("<div class='mmc3-label'>Storage location</div>"), storage_type]),
    ],
    layout=w.Layout(grid_template_columns="repeat(2, minmax(200px, 1fr))", grid_gap="10px", width='100%')
)

# Handling & logistics (no horizontal scrollbar)
for cb in (tower, telesc, teleh, flatbed, aframe):
    cb.layout = w.Layout(width='auto')
    cb.style = {'description_width': 'initial'}

handling_grid = w.GridBox(
    children=[tower, telesc, teleh, flatbed, aframe],
    layout=w.Layout(
        grid_template_columns="repeat(auto-fit, minmax(180px, 1fr))",
        grid_auto_rows="min-content",
        grid_gap="6px",
        width='100%',
        overflow_x='hidden'
    )
)
row_handling = w.VBox([
    w.HTML("<div class='mmc3-label'>Handling & logistics</div>"),
    handling_grid
])

row_geometry = w.GridBox(
    children=[
        w.VBox([w.HTML("<div class='mmc3-label'>Panel height need</div>"), panel_h_need]),
        w.VBox([w.HTML("<div class='mmc3-label'>Frame length need</div>"), frame_len_need]),
        w.VBox([w.HTML("<div class='mmc3-label'>Frame width &gt; 3.2 m?</div>"), width_over]),
    ],
    layout=w.Layout(grid_template_columns="repeat(3, minmax(200px, 1fr))", grid_gap="8px", width='100%')
)

toolbar = w.HBox([
    w.VBox([w.HTML("<div class='mmc3-title'>MMC Viability Assessment</div>"), status_html],
           layout=w.Layout(flex='1 1 auto')),
    w.HBox([apply_btn, export_btn, reset_btn], layout=w.Layout(align_items='center', gap='6px'))
], layout=w.Layout(width='100%', align_items='center'), _dom_classes=['mmc3-toolbar'])

results_section = w.VBox([w.HTML("<hr class='mmc3-hr'>"), out], layout=w.Layout(width='100%'))

card = w.VBox([toolbar, row_primary, row_handling, row_geometry, results_section],
              layout=w.Layout(width='100%'),
              _dom_classes=['mmc3-app','wb-skin'])

# Optional translucency if you have as_glass() defined
try:
    as_glass(card, elev=5)  # noqa: F821
except Exception:
    pass

### Viability Assessment UI

In [20]:
display(card)

# ===================== 4) Optional handoff to selection UI =====================
ACTIVE_VIABLE_METHODS = None
def _norm_key(s): return str(s).strip().casefold()

def _apply_viability_list(viable_list):
    if 'df_active_full' not in globals(): return
    keyset = {_norm_key(x) for x in (viable_list or [])}
    if keyset:
        mask = globals()['df_active_full']['system_type'].astype(str).map(_norm_key).isin(keyset)
        globals()['df_active'] = globals()['df_active_full'].loc[mask].copy()
    else:
        globals()['df_active'] = globals()['df_active_full'].copy()

    if 'sys_type_dd' in globals():
        current = sys_type_dd.value
        opts = [''] + sorted(globals()['df_active']['system_type'].dropna().astype(str).unique().tolist())
        sys_type_dd.options = opts
        sys_type_dd.value = current if current in opts else ''
        if 'refresh_all' in globals(): refresh_all()

def set_viable_methods_from_results(results_df):
    global ACTIVE_VIABLE_METHODS
    if results_df is None or results_df.empty: return
    ACTIVE_VIABLE_METHODS = (results_df.loc[results_df['Viable'] == True, 'MMC Method']
                             .dropna().astype(str).tolist())
    _apply_viability_list(ACTIVE_VIABLE_METHODS)

def reapply_viability_filter():
    _apply_viability_list(ACTIVE_VIABLE_METHODS)

if '_latest_results' in globals() and _latest_results is not None:
    set_viable_methods_from_results(_latest_results)

## MMC Generic Model Assessment

### Backend

In [21]:
# ==== Cell 2 — MMC Generic Model Assessment (whiteboard-styled) ====
# (System Category = Wall/Cladding/Slab only; transparent visuals; frameless charts; no stray sliders)
import re, warnings, numpy as np, pandas as pd, ipywidgets as w
from pathlib import Path
import IPython.display as IPD
from IPython.display import display as ipy_display, HTML, clear_output
from openpyxl import load_workbook
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from collections import Counter

# Un-shadow display() if needed
if 'display' in globals() and globals()['display'] is not IPD.display:
    try: del globals()['display']
    except Exception: pass

warnings.filterwarnings("ignore", category=UserWarning)

# Matplotlib: transparent backgrounds by default
plt.rcParams['figure.facecolor']  = 'none'
plt.rcParams['axes.facecolor']    = 'none'
plt.rcParams['savefig.facecolor'] = 'none'

# ===================== Data source ===========================
SRC = Path("ireland_generic_mmc_model.xlsx")
if not SRC.exists():
    alt = Path("/mnt/data/ireland_generic_mmc_model.xlsx")
    if alt.exists(): SRC = alt
    else: raise FileNotFoundError("ireland_generic_mmc_model.xlsx not found in working dir or /mnt/data.")

# =================== Helpers / loading =======================
def _norm(s):
    if s is None: return ""
    return re.sub(r"[^a-z0-9]+","",str(s).strip().lower())

def _coerce_num(x):
    if x is None or (isinstance(x,float) and np.isnan(x)): return np.nan
    s = str(x).strip()
    if s == "" or s.lower()=="nan": return np.nan
    try: return float(s)
    except: pass
    if re.match(r"^\d{1,3}(\.\d{3})+,\d+$", s): return float(s.replace('.','').replace(',','.'))
    if re.match(r"^\d{1,3}(,\d{3})+(\.\d+)?$", s): return float(s.replace(',',''))
    if re.match(r"^\d+,\d+$", s): return float(s.replace(',','.'))
    s2 = re.sub(r"[^0-9eE\.\-\+]", "", s)
    try: return float(s2)
    except: return np.nan

def _read_sheet_cached(path: Path, sheet_name: str) -> pd.DataFrame:
    wb = load_workbook(path, data_only=True, read_only=True)
    if sheet_name not in wb.sheetnames: return pd.DataFrame()
    ws = wb[sheet_name]
    rows = list(ws.iter_rows(values_only=True))
    if not rows: return pd.DataFrame()
    header = None; data_start = 0
    for i, r in enumerate(rows):
        if r and any(v is not None for v in r):
            header = [str(x).strip() if x is not None else "" for x in r]; data_start = i + 1; break
    if header is None: return pd.DataFrame()
    df = pd.DataFrame(rows[data_start:], columns=header).dropna(how="all")
    return df

def _find_col(df, names, must=False):
    targets = [_norm(n) for n in names]
    for c in df.columns:
        nc = _norm(c)
        for t in targets:
            if t and t in nc: return c
    if must: raise KeyError(f"Could not find any of {names} in columns: {list(df.columns)}")
    return None

def _derive_sheet_metrics(df: pd.DataFrame) -> pd.DataFrame:
    d = df.copy()
    c_mass = _find_col(d, ["Mass (kg/m²)","Mass kg/m2","Mass kg/m^2"])
    c_cf   = _find_col(d, ["Carbon Factor","Carbon factor (kgCO2e/kg)"])
    c_a1a3 = _find_col(d, ["A1–A3 (kgCO₂e/m²)","A1-A3 (kgCO2e/m2)"])
    for cc in [c_mass, c_cf, c_a1a3]:
        if cc and cc in d.columns: d[cc] = d[cc].map(_coerce_num)
    if c_a1a3 and c_mass and c_cf:
        miss = d[c_a1a3].isna()
        d.loc[miss, c_a1a3] = d.loc[miss, c_mass] * d.loc[miss, c_cf]
    return d

def load_all_sheets(path: Path) -> dict:
    wb = load_workbook(path, read_only=True)
    sheets = {}
    for name in wb.sheetnames:
        try: df = _read_sheet_cached(path, name)
        except: df = pd.read_excel(path, sheet_name=name)
        if df.empty: continue
        df = _derive_sheet_metrics(df)
        df["__sheet__"] = name
        sheets[name] = df
    if not sheets: raise RuntimeError("No non-empty sheets found.")
    return sheets

SHEETS = load_all_sheets(SRC)

# ================= Canonical categories & aliasing =================
SYS_TYPE_ALLOWED = {"Wall Systems":"wall","Cladding Systems":"cladding","Slab Systems":"slab"}
MMC_CAT_ALLOWED  = {"Concrete Block","ICF (Insulating Concrete Formwork)","LGS (Light Gauge Steel)","Timber"}

SYS_TYPE_SYNONYMS = {
    "wallsystems":"Wall Systems","wall":"Wall Systems",
    "claddingsystems":"Cladding Systems","cladding":"Cladding Systems",
    "slabsystems":"Slab Systems","slab":"Slab Systems",
}
MMC_METHOD_SYNONYMS = {
    "concreteblock":"Concrete Block","block":"Concrete Block",
    "icf":"ICF (Insulating Concrete Formwork)","insulatingconcreteformwork":"ICF (Insulating Concrete Formwork)",
    "lightgaugesteel":"LGS (Light Gauge Steel)","lgs":"LGS (Light Gauge Steel)",
    "timber":"Timber","wood":"Timber",
}

def _canon_choice(val: str, synonyms: dict, allowed=None):
    s = _norm(val)
    if s in synonyms: return synonyms[s]
    if isinstance(val, str):
        v = val.strip()
        if allowed and v in (allowed if isinstance(allowed, (set, list, tuple)) else allowed.keys()):
            return v
    return None

ALIAS = {
    "LayerNo": ["Layer No.","LayerNo","Layer Number","Layer #"],
    "GenericMaterial": ["Generic Material","Material","Generic Material Name"],
    "FunctionalRole": ["Functional Role","Role","Function"],
    "Mass_per_m2_kg": ["Mass (kg/m²)","Mass kg/m2","Mass kg/m^2"],
    "CarbonFactor_kgCO2e_per_kg": ["Carbon Factor","Carbon factor (kgCO2e/kg)","GWP factor (kgCO2e/kg)"],
    "A1A3_kgCO2e_per_m2": ["A1–A3 (kgCO₂e/m²)","A1-A3 (kgCO2e/m2)"],
    "A1A3_kgCO2e_per_5_76m2": ["A1–A3 (kg CO₂e / 5.76 m²)","A1-A3 (kgCO2e / 5.76 m2)"],
    "Mass_per_5_76m2_kg": ["Mass (kg / 5.76 m²)","Mass kg / 5.76 m2"],
    "CarbonFactor": ["Carbon Factor"],
    "CFUnit": ["Carbon Factor Unit","CF unit","Unit"],
    "Total_kgCO2e": ["Total GWP (kgCO₂e)","Total kgCO2e","Total GWP (kgCO2e)"],
    "ThermalConductivity": ["Thermal Conductivity (W/m·K)","Lambda","k-value"],
    "RValue": ["R-Value (m²·K/W)","R-Value (m2K/W)","R Value"],
    "UValue": ["U-Value (W/m²·K)","U-Value (W/m2K)","U Value"],
    "LifeExpectancy_years": ["Life Expectancy (years)","Service life (years)"],
    "SystemID": ["System ID","SystemID","ID"],
    "SystemName": ["System Name","SystemName","Name"],
    "SystemCategory": ["System Category","SystemCategory","System Type","SystemType"],
    "MMCMethod": ["MMC Method","MMCMethod","MMC method","Method"],
}

def _find_alias_main(df, key): return _find_col(df, ALIAS.get(key, [key]))

def _derive_system_type_key_from_df(df: pd.DataFrame, fallback_sheet_name: str = "") -> str:
    col = _find_alias_main(df, "SystemCategory")
    if col and col in df.columns and df[col].notna().any():
        mapped = []
        for v in df[col].astype(str).dropna():
            cho = _canon_choice(v, SYS_TYPE_SYNONYMS, allowed=SYS_TYPE_ALLOWED)
            if cho: mapped.append(cho)
        if mapped:
            most = Counter(mapped).most_common(1)[0][0]
            return SYS_TYPE_ALLOWED.get(most, "generic")
    s = _norm(fallback_sheet_name)
    if "slab" in s: return "slab"
    if "clad" in s: return "cladding"
    if "wall" in s or "wll" in s: return "wall"
    return "generic"

# =================== Formatting helpers =====================
def _safe_text(val):
    if val is None: return ""
    try:
        if isinstance(val, float) and np.isnan(val): return ""
    except Exception: pass
    s = str(val).strip()
    return "" if s.lower() == "nan" else s

def _fmt_smart(val):
    if val is None: return "-"
    try:
        v = float(val)
        if np.isnan(v) or np.isinf(v): return "-"
        if abs(v - round(v)) < 1e-9:   return f"{int(round(v))}"
        return f"{v:.2f}"
    except Exception:
        return str(val).strip() or "-"

def _sum(df, col): return df[col].sum(skipna=True) if (col in df and not df[col].isna().all()) else np.nan

def _mass_weighted_cf(df, cf_col_key="Carbon Factor", mass_col_key="Mass (kg/m²)"):
    cfcol = _find_alias_main(df, cf_col_key); mcol = _find_alias_main(df, mass_col_key)
    if not (cfcol and mcol): return np.nan
    m = pd.to_numeric(df[mcol], errors="coerce"); cf = pd.to_numeric(df[cfcol], errors="coerce")
    if m.fillna(0).sum() == 0: return np.nan
    return (cf.fillna(0) * m.fillna(0)).sum() / m.fillna(0).sum()

def _unique_scalar(df, col):
    if not col or col not in df.columns: return np.nan
    s = df[col].dropna()
    if s.empty: return np.nan
    vals = pd.to_numeric(s, errors="coerce")
    if vals.notna().all() and vals.max() - vals.min() < 1e-9: return vals.iloc[0]
    t = s.astype(str).str.strip().unique()
    return s.iloc[0] if len(t)==1 else np.nan

def _get_sys_name(df_sys: pd.DataFrame, default: str) -> str:
    name_col = _find_alias_main(df_sys, "SystemName")
    if name_col and not df_sys.empty and df_sys[name_col].notna().any():
        return _safe_text(df_sys[name_col].dropna().astype(str).iloc[0])
    return default

# ================== A4 (transport) ===================
A4_SHEET_CANDIDATES = ["Wall Systems (A4)", "Walls (A4)", "Wall A4", "A4 Walls", "A4"]
A4_ALIASES = {
    "System ID": ["System ID","SystemID","ID"],
    "System Name": ["System Name","SystemName","Name"],
    "Distance Traveled (km)": ["Distance Traveled (km)","DistanceTraveled_km","Distance (km)","km"],
    "Total per Truck (kgCO₂e)": ["Total per Truck (kgCO₂e)","kgCO2e_per_Truck","kgCO₂e per Truck"],
    "A4 (kgCO₂e/m²)": ["A4 (kgCO₂e/m²)","A4_kgCO2e_per_m2","A4 kgCO2e/m2"]
}
def _find_alias_a4(df, key):
    targets = [re.sub(r"[^a-z0-9]+","", s.lower()) for s in A4_ALIASES.get(key, [key])]
    for c in df.columns:
        nc = re.sub(r"[^a-z0-9]+","", str(c).lower())
        if any(t in nc for t in targets): return c
    return None

def _load_a4_table(path: Path) -> pd.DataFrame:
    try: wb = load_workbook(path, data_only=True, read_only=True)
    except Exception: return pd.DataFrame(columns=["System ID","System Name","A4 (kgCO₂e/m²)","Distance Traveled (km)"])
    sheet = None
    for cand in A4_SHEET_CANDIDATES:
        for s in wb.sheetnames:
            if re.sub(r"[^a-z0-9]+","", cand.lower()) in re.sub(r"[^a-z0-9]+","", s.lower()):
                sheet = s; break
        if sheet: break
    if not sheet: return pd.DataFrame(columns=["System ID","System Name","A4 (kgCO₂e/m²)","Distance Traveled (km)"])
    try: raw = _read_sheet_cached(path, sheet)
    except Exception: raw = pd.read_excel(path, sheet_name=sheet)
    if raw.empty: return pd.DataFrame(columns=["System ID","System Name","A4 (kgCO₂e/m²)","Distance Traveled (km)"])
    col_id    = _find_alias_a4(raw, "System ID")
    col_name  = _find_alias_a4(raw, "System Name")
    col_a4    = _find_alias_a4(raw, "A4 (kgCO₂e/m²)")
    col_km    = _find_alias_a4(raw, "Distance Traveled (km)")
    col_truck = _find_alias_a4(raw, "Total per Truck (kgCO₂e)")
    keep = [c for c in [col_id, col_name, col_a4, col_km, col_truck] if c]
    if not keep: return pd.DataFrame(columns=["System ID","System Name","A4 (kgCO₂e/m²)","Distance Traveled (km)"])
    df = raw[keep].copy()
    ren = {}
    if col_id:    ren[col_id]    = "System ID"
    if col_name:  ren[col_name]  = "System Name"
    if col_a4:    ren[col_a4]    = "A4 (kgCO₂e/m²)"
    if col_km:    ren[col_km]    = "Distance Traveled (km)"
    if col_truck: ren[col_truck] = "Total per Truck (kgCO₂e)"
    df = df.rename(columns=ren)
    for c in ["A4 (kgCO₂e/m²)","Distance Traveled (km)","Total per Truck (kgCO₂e)"]:
        if c in df.columns: df[c] = df[c].map(_coerce_num)
    if "A4 (kgCO₂e/m²)" not in df.columns: df["A4 (kgCO₂e/m²)"] = np.nan
    if "Total per Truck (kgCO₂e)" in df.columns:
        need = df["A4 (kgCO₂e/m²)"].isna()
        df.loc[need, "A4 (kgCO₂e/m²)"] = df.loc[need, "Total per Truck (kgCO₂e)"] / 500.0
    df = df.dropna(how="all")
    for c in ["System ID","System Name","A4 (kgCO₂e/m²)","Distance Traveled (km)"]:
        if c not in df.columns: df[c] = np.nan
    return df[["System ID","System Name","A4 (kgCO₂e/m²)","Distance Traveled (km)"]]

A4_TABLE = _load_a4_table(SRC)
def _norm_key_join(s: str) -> str: return re.sub(r"[^a-z0-9]+","", str(s or "").strip().lower())
A4_BY_ID, A4_BY_NAME = {}, {}
if not A4_TABLE.empty:
    for _, r in A4_TABLE.iterrows():
        a4_val = _coerce_num(r.get("A4 (kgCO₂e/m²)"))
        if a4_val is None or not np.isfinite(a4_val): continue
        km = _coerce_num(r.get("Distance Traveled (km)"))
        if pd.notna(r.get("System ID")):   A4_BY_ID[_norm_key_join(r["System ID"])]   = dict(A4=float(a4_val), km=(None if not np.isfinite(km) else float(km)))
        if pd.notna(r.get("System Name")): A4_BY_NAME[_norm_key_join(r["System Name"])] = dict(A4=float(a4_val), km=(None if not np.isfinite(km) else float(km)))

def _lookup_a4_for_selection(df_sys: pd.DataFrame, sheet_name: str):
    cat_key = _derive_system_type_key_from_df(df_sys, sheet_name)
    if cat_key != "wall": return None
    if df_sys.empty: return None
    id_col = _find_alias_main(df_sys, "SystemID")
    if id_col and df_sys[id_col].notna().any():
        sid = _norm_key_join(df_sys[id_col].dropna().astype(str).iloc[0])
        if sid in A4_BY_ID: return A4_BY_ID[sid]
    name_col = _find_alias_main(df_sys, "SystemName")
    if name_col and df_sys[name_col].notna().any():
        nm = _norm_key_join(df_sys[name_col].dropna().astype(str).iloc[0])
        if nm in A4_BY_NAME: return A4_BY_NAME[nm]
    return None

# ====================== KPI builders ========================
def kpis_wall(df, sheet_name, include_a4=True):
    k = [("Layers (#)", len(df))]
    mass_col = _find_alias_main(df, "Mass_per_m2_kg")
    cf_col   = _find_alias_main(df, "CarbonFactor_kgCO2e_per_kg")
    a1a3_m2  = _find_alias_main(df, "A1A3_kgCO2e_per_m2")
    a1a3_576 = _find_alias_main(df, "A1A3_kgCO2e_per_5_76m2")
    mass_m2 = _sum(df, mass_col) if mass_col else np.nan
    cf_wavg = _mass_weighted_cf(df) if cf_col else np.nan
    a1a3_m2_val = _sum(df, a1a3_m2) if a1a3_m2 else np.nan
    if mass_col: k.append(("Mass (kg/m²)", _fmt_smart(mass_m2)))
    if cf_col:   k.append(("Carbon factor (kg CO₂e/kg)", _fmt_smart(cf_wavg)))
    if a1a3_576: k.append(("A1–A3 (kg CO₂e / 5.76 m²)", _fmt_smart(_sum(df, a1a3_576))))
    if a1a3_m2:  k.append(("A1–A3 (kg CO₂e / m²)", _fmt_smart(a1a3_m2_val)))
    a4 = _lookup_a4_for_selection(df, sheet_name) if include_a4 else None
    if include_a4 and a4 is not None and np.isfinite(a4.get("A4", np.nan)):
        a4_val = float(a4["A4"])
        if np.isfinite(a1a3_m2_val):
            k.append(("A4 (transport) (kg CO₂e / m²)", _fmt_smart(a4_val)))
            k.append(("A1–A4 (kg CO₂e / m²)", _fmt_smart(a1a3_m2_val + a4_val)))
        else:
            k.append(("A4 (transport) (kg CO₂e / m²)", _fmt_smart(a4_val)))
    return "Wall — Key Performance Indicators", k

def kpis_cladding(df, *_):
    k = [("Layers (#)", len(df))]
    mass_col = _find_alias_main(df, "Mass_per_m2_kg")
    cf_col   = _find_alias_main(df, "CarbonFactor")
    cfu_col  = _find_alias_main(df, "CFUnit")
    tot_col  = _find_alias_main(df, "Total_kgCO2e")
    lam_col  = _find_alias_main(df, "ThermalConductivity")
    r_col    = _find_alias_main(df, "RValue")
    u_col    = _find_alias_main(df, "UValue")
    life_col = _find_alias_main(df, "LifeExpectancy_years")
    if mass_col: k.append(("Mass (kg/m²)", _fmt_smart(_sum(df, mass_col))))
    if cf_col:
        unit = str(_unique_scalar(df, cfu_col)).lower()
        if "kgco2e/kg" in unit.replace(" ", ""):
            wavg = _mass_weighted_cf(df, cf_col_key="CarbonFactor", mass_col_key="Mass (kg/m²)")
            k.append(("Carbon factor (kg CO₂e/kg)", _fmt_smart(wavg)))
        else:
            k.append(("Carbon factor", _fmt_smart(_unique_scalar(df, cf_col))))
    if cfu_col:  k.append(("CF unit", _safe_text(_unique_scalar(df, cfu_col)) or "-"))
    if tot_col:  k.append(("Total (kg CO₂e)", _fmt_smart(_sum(df, tot_col))))
    if lam_col:  k.append(("Thermal conductivity (W/m·K)", _fmt_smart(_unique_scalar(df, lam_col))))
    if r_col:    k.append(("R-value (m²·K/W)", _fmt_smart(_unique_scalar(df, r_col))))
    if u_col:    k.append(("U-value (W/m²·K)", _fmt_smart(_unique_scalar(df, u_col))))
    if life_col: k.append(("Life expectancy (years)", _fmt_smart(_unique_scalar(df, life_col))))
    return "Cladding — Key Performance Indicators", k

def kpis_slab(df, *_):
    k = [("Layers (#)", len(df))]
    mass_576 = _find_alias_main(df, "Mass_per_5_76m2_kg")
    cf_col   = _find_alias_main(df, "CarbonFactor_kgCO2e_per_kg")
    a1a3_576 = _find_alias_main(df, "A1A3_kgCO2e_per_5_76m2")
    a1a3_m2  = _find_alias_main(df, "A1A3_kgCO2e_per_m2")
    if mass_576: k.append(("Mass (kg / 5.76 m²)", _fmt_smart(_sum(df, mass_576))))
    if cf_col:   k.append(("Carbon factor (kg CO₂e/kg)", _fmt_smart(_mass_weighted_cf(df, cf_col_key="Carbon Factor", mass_col_key="Mass (kg/m²)"))))
    if a1a3_576: k.append(("A1–A3 (kg CO₂e / 5.76 m²)", _fmt_smart(_sum(df, a1a3_576))))
    if a1a3_m2:  k.append(("A1–A3 (kg CO₂e / m²)", _fmt_smart(_sum(df, a1a3_m2))))
    return "Slab — Key Performance Indicators", k

def _category_for_df(df: pd.DataFrame, sheet_name: str):
    return _derive_system_type_key_from_df(df, sheet_name)

def kpis_for_sheet(sheet_name: str, df_sys: pd.DataFrame, include_a4=True):
    cat = _category_for_df(df_sys, sheet_name)
    if cat == "wall":     return kpis_wall(df_sys, sheet_name, include_a4)
    if cat == "cladding": return kpis_cladding(df_sys)
    if cat == "slab":     return kpis_slab(df_sys)
    return "System — KPIs", [("Layers (#)", len(df_sys))]

# ======== KPI rendering (hide NaN/zero) =====================
def _render_kpi_cards(system_name, subtitle, kpis, theme, a4_note=None):
    def _is_meaningful(value):
        if value is None: return False
        if isinstance(value, (int, float, np.floating)):
            return not (pd.isna(value) or float(value) == 0.0)
        s = str(value).strip()
        if s in ("", "-", "nan", "NaN", "None"): return False
        try:
            v = float(s.replace(",", ""))
            if np.isnan(v) or v == 0.0: return False
        except Exception: pass
        return True
    filtered = [(label, value) for (label, value) in kpis if _is_meaningful(value)]
    heading_html = f"""
    <div style="margin:8px 0 10px 0">
      <div style="font-size:16px;font-weight:800;color:{theme['heading']}">{subtitle}</div>
      <div style="font-size:13px;color:#555">{system_name}</div>
      {f'<div class="small" style="color:#666;margin-top:4px">{a4_note}</div>' if a4_note else ''}
    </div>"""
    if not filtered:
        ipy_display(HTML(heading_html)); return
    cards = []
    for label, value in filtered:
        cards.append(f"""
        <div class="wb-card" style="padding:10px 14px;min-width:190px;flex:0 0 auto;">
          <div style="font-size:12px;color:#666">{label}</div>
          <div style="font-size:22px;font-weight:800;color:{theme['heading']}">{value}</div>
        </div>""")
    strip = "<div style='display:flex;gap:10px;flex-wrap:nowrap;align-items:stretch;margin:4px 0 10px 0'>" + "".join(cards) + "</div>"
    ipy_display(HTML(heading_html + strip))

# =================== Table rendering =========================
def _allowed_metric_columns_for_table(df: pd.DataFrame, sheet_name: str):
    cat = _category_for_df(df, sheet_name)
    if cat == "wall":
        return ["Mass_per_m2_kg","CarbonFactor_kgCO2e_per_kg","A1A3_kgCO2e_per_5_76m2","A1A3_kgCO2e_per_m2"]
    if cat == "cladding":
        return ["Mass_per_m2_kg","CarbonFactor","CFUnit","Total_kgCO2e","ThermalConductivity","RValue","UValue","LifeExpectancy_years"]
    if cat == "slab":
        return ["Mass_per_5_76m2_kg","CarbonFactor_kgCO2e_per_kg","A1A3_kgCO2e_per_5_76m2","A1A3_kgCO2e_per_m2"]
    return []

def _collect_present_cols(df, logical_names):
    present = []
    for key in logical_names:
        col = _find_alias_main(df, key)
        if col: present.append(col)
    return present

def _styled_layers(df_layers, heading, sheet_name):
    identity_exclude = {"SystemCategory","MMCMethod","SystemID","SystemName","SourceHeader","__sheet__"}
    layer_cols = [c for c in [_find_alias_main(df_layers, "LayerNo"),
                              _find_alias_main(df_layers, "FunctionalRole"),
                              _find_alias_main(df_layers, "GenericMaterial")] if c]
    metric_keys = _allowed_metric_columns_for_table(df_layers, sheet_name)
    metric_cols = _collect_present_cols(df_layers, metric_keys)
    ordered = [c for c in (layer_cols + metric_cols) if c and c in df_layers.columns and c not in identity_exclude]
    if not ordered:
        ipy_display(HTML(f"<h4 style='margin:6px 0 6px 0'>{heading}</h4><div style='color:#777'>No displayable columns.</div>"))
        return
    dfv = df_layers[ordered].copy()
    def _is_numericish(col):
        nc = _norm(col)
        return (pd.api.types.is_numeric_dtype(dfv[col])
                or any(k in nc for k in ["kgco2e","mass","a1a3","uvalue","rvalue","thermal","life"]))
    num_cols = [c for c in dfv.columns if _is_numericish(c)]
    sty = dfv.style.format({c: _fmt_smart for c in num_cols}, na_rep="-")
    # transparent table styles
    sty = (sty.set_table_styles([
        {'selector':'','props':[('background-color','transparent')]},
        {'selector':'table','props':[('background-color','transparent')]},
        {'selector':'thead','props':[('background-color','transparent')]},
        {'selector':'tbody','props':[('background-color','transparent')]},
        {'selector':'th','props':[('background-color','transparent')]},
        {'selector':'td','props':[('background-color','transparent')]}
    ], overwrite=False).set_properties(**{'background-color':'transparent'}))
    ipy_display(HTML(f"<h4 style='margin:6px 0 6px 0'>{heading}</h4>"))
    try: ipy_display(sty.hide(axis="index"))
    except Exception: ipy_display(sty.hide_index())

# ============= Chart utilities ================
def _strip_axes(ax, keep_grid=True):
    """Remove plot frame/spines; keep optional light grid."""
    for side in ["top", "right", "left", "bottom"]:
        ax.spines[side].set_visible(False)
    ax.set_frame_on(False)
    ax.tick_params(left=False, bottom=False)
    if keep_grid: ax.grid(True, alpha=0.2)
    ax.figure.patch.set_alpha(0)
    ax.set_facecolor('none')

def _gradient_from_values(base_hex, values, t_min=0.35, t_max=0.95):
    base = np.array(mcolors.to_rgb(base_hex)); white = np.ones(3)
    vals = np.asarray(values, dtype=float)
    if len(vals) == 0: return []
    vmin = np.nanmin(vals); vmax = np.nanmax(vals)
    if not np.isfinite(vmin) or not np.isfinite(vmax): vmin, vmax = 0.0, 1.0
    if vmax - vmin < 1e-12:
        ts = np.linspace((t_min + t_max) / 2.0, t_max, len(vals))
    else:
        norm = (vals - vmin) / (vmax - vmin)
        ts = t_min + (t_max - t_min) * norm
    return [tuple(white * (1 - t) + base * t) for t in ts]

def _charts_for(df_layers, theme):
    lno = _find_alias_main(df_layers, "LayerNo")
    if not lno or lno not in df_layers.columns: return
    x = pd.to_numeric(df_layers[lno], errors="coerce")
    if x.isna().all(): return
    a1a3_col = _find_alias_main(df_layers, "A1A3_kgCO2e_per_m2") or _find_alias_main(df_layers, "A1A3_kgCO2e_per_5_76m2")
    mass_col = _find_alias_main(df_layers, "Mass_per_m2_kg") or _find_alias_main(df_layers, "Mass_per_5_76m2_kg")
    left = w.Output(); right = w.Output()
    ipy_display(w.HBox([left, right]))
    with left:
        if a1a3_col:
            y = pd.to_numeric(df_layers[a1a3_col], errors="coerce").fillna(0).values
            colors = _gradient_from_values(theme["bar"], y)
            fig1, ax1 = plt.subplots(figsize=(6,3.2))
            ax1.bar(x, y, color=colors, edgecolor=theme["bar"])
            ax1.set_xlabel("Layer No"); ax1.set_ylabel(a1a3_col.replace("_"," ").replace("576","5.76"))
            ax1.set_title("Per-Layer A1–A3 (gradient by impact)")
            _strip_axes(ax1, keep_grid=True)
            plt.show()
    with right:
        if mass_col:
            y2 = pd.to_numeric(df_layers[mass_col], errors="coerce").fillna(0).values
            colors2 = _gradient_from_values(theme["bar"], y2)
            fig2, ax2 = plt.subplots(figsize=(6,3.2))
            ax2.bar(x, y2, color=colors2, edgecolor=theme["bar"])
            ax2.set_xlabel("Layer No"); ax2.set_ylabel(mass_col.replace("_"," ").replace("576","5.76"))
            ax2.set_title("Per-Layer Mass (gradient by contribution)")
            _strip_axes(ax2, keep_grid=True)
            plt.show()

# ================= Filtering & slicing =======================
def _get_mmc_method_col(df: pd.DataFrame) -> str | None:
    return _find_alias_main(df, "MMCMethod")

def systems_for_filters(df_sheet, mmc_value):
    d = df_sheet.copy()
    mmc_col = _get_mmc_method_col(d)
    if mmc_value and mmc_col:
        mmc_norm = []
        series = d[mmc_col].astype(str)
        for v in series:
            cho = _canon_choice(v, MMC_METHOD_SYNONYMS, allowed=MMC_CAT_ALLOWED)
            mmc_norm.append(cho if cho else None)
        mask = [x == mmc_value for x in mmc_norm]
        d = d.loc[mask]
    id_col = _find_alias_main(d, "SystemID")
    name_col = _find_alias_main(d, "SystemName")
    if not id_col:
        return [("[SYNTH] — (entire sheet)", "__ALL__")]
    cols = [id_col] + ([name_col] if name_col else [])
    ids = d[cols].dropna(subset=[id_col]).drop_duplicates()
    def _label(r):
        sid = _safe_text(r[id_col])
        sname = _safe_text(r[name_col]) if (name_col and name_col in r.index) else ""
        return f"[{sid}] — {sname}" if sname else f"[{sid}]"
    return [(_label(r), r[id_col]) for _, r in ids.iterrows()]

def slice_system(df_sheet, mmc_value, system_id):
    d = df_sheet.copy()
    mmc_col = _get_mmc_method_col(d)
    if mmc_value and mmc_col:
        mmc_norm = []
        for v in d[mmc_col].astype(str):
            cho = _canon_choice(v, MMC_METHOD_SYNONYMS, allowed=MMC_CAT_ALLOWED)
            mmc_norm.append(cho if cho else None)
        d = d.loc[[x == mmc_value for x in mmc_norm]]
    id_col = _find_alias_main(d, "SystemID")
    if system_id == "__ALL__" or not id_col:
        out = d.copy()
    else:
        out = d[d[id_col] == system_id].copy()
    lno = _find_alias_main(out, "LayerNo")
    if lno and lno in out.columns:
        out[lno] = pd.to_numeric(out[lno], errors="coerce")
        sort_cols = [c for c in [lno, _find_alias_main(out, "GenericMaterial")] if c]
        out = out.sort_values(by=sort_cols, kind="mergesort")
    return out

# ===================== Advanced comparison helpers ======================
def _has_finite(x):
    try: return (x is not None) and np.isfinite(float(x))
    except Exception: return False

def _plot_tradeoff_mass_vs_carbon(ax, sa, sb, use_a4, color_a, color_b):
    """Frameless scatter with safe in-bounds guides + A=B handling."""
    y_key = "a1a4" if use_a4 else "a1a3"
    y_lab = "A1–A4 (kg CO₂e / m²)" if use_a4 else "A1–A3 (kg CO₂e / m²)"

    xa, ya = float(sa["mass"]), float(sa[y_key])
    xb, yb = float(sb["mass"]), float(sb[y_key])

    xs = [xa, xb]; ys = [ya, yb]
    xr = max(1e-6, max(xs) - min(xs))
    yr = max(1e-6, max(ys) - min(ys))
    x_min = max(0, min(xs) - xr * 0.10); x_max = max(xs) + xr * 0.10
    y_min = max(0, min(ys) - yr * 0.15); y_max = max(ys) + yr * 0.22

    ax.set_xlim(x_min, x_max); ax.set_ylim(y_min, y_max)
    ax.set_xlabel("Mass (kg / m²)"); ax.set_ylabel(y_lab)

    # In-bounds isocarbon-intensity guide lines (kgCO2e/kg)
    guide_cf = [0.05, 0.10, 0.20]
    for c in guide_cf:
        y1, y2 = c * x_min, c * x_max
        if (y1 <= y_max) and (y2 >= y_min):
            ax.plot([x_min, x_max], [y1, y2], linestyle="--", linewidth=1, alpha=0.25, color="#666")
            y_lab_pos = float(np.clip(c * x_max, y_min, y_max))
            ax.text(x_max, y_lab_pos, f"{c:.2f} kgCO₂e/kg",
                    ha="right", va="bottom", fontsize=8, color="#666", clip_on=True)

    # Points + label/arrow
    same_point = (abs(xa - xb) + abs(ya - yb)) < 1e-6
    ax.scatter([xa], [ya], s=90, color=color_a, edgecolors="white", linewidth=1.5, zorder=3)
    if same_point:
        ax.scatter([xb], [yb], s=140, facecolors="none", edgecolors=color_b, linewidth=2.0, zorder=3)
        ax.text(xa, ya, f"  A = B ({ya:.1f})", va="center", fontsize=9, color="#333")
    else:
        ax.scatter([xb], [yb], s=90, color=color_b, edgecolors="white", linewidth=1.5, zorder=3)
        ax.annotate("", xy=(xb, yb), xytext=(xa, ya),
                    arrowprops=dict(arrowstyle="->", lw=2, color="#444", alpha=0.85))
        ax.text(xa, ya, f"  A ({ya:.1f})", va="center", fontsize=9, color="#333")
        ax.text(xb, yb, f"  B ({yb:.1f})", va="center", fontsize=9, color="#333")

    ax.set_title("Carbon vs Mass — trade-off (arrow A→B)", pad=10)
    _strip_axes(ax, keep_grid=True)
    ax.figure.tight_layout(pad=0.8)

def _plot_waterfall_a1a4(ax, sa, sb, color_pos="#74C69D", color_neg="#E5989B"):
    """Frameless A→B waterfall with generous headroom and neat labels."""
    A_total = float(sa["a1a3"]) + float(sa.get("a4", 0.0))
    B_total = float(sb["a1a3"]) + float(sb.get("a4", 0.0))
    d_a1a3  = float(sb["a1a3"]) - float(sa["a1a3"])
    d_a4    = float(sb.get("a4", 0.0)) - float(sa.get("a4", 0.0))

    steps   = ["A total", "Δ A1–A3", "Δ A4", "B total"]
    values  = [A_total, d_a1a3, d_a4, B_total]
    cum     = [0, A_total, A_total + d_a1a3, 0]

    colors = ["#95D5B2",
              (color_pos if d_a1a3 >= 0 else color_neg),
              (color_pos if d_a4   >= 0 else color_neg),
              "#74B5FF"]

    ax.bar(range(4),
           [values[0], values[1], values[2], values[3]],
           bottom=[0, cum[1], cum[2], 0],
           color=colors, edgecolor="#ddd")

    candidates = [A_total, B_total, cum[1], cum[2], cum[1] + values[1], cum[2] + values[2]]
    y_min = min(0, min(candidates))
    y_max = max(candidates)
    pad   = max(1.0, (y_max - y_min) * 0.22)
    ax.set_ylim(y_min - pad * 0.15, y_max + pad)

    ax.plot([0, 1], [values[0], cum[1]], color="#999", lw=1)
    ax.plot([1, 2], [cum[1] + values[1], cum[2]], color="#999", lw=1)

    off = (ax.get_ylim()[1] - ax.get_ylim()[0]) * 0.03
    for i, v in enumerate(values):
        is_change = i in (1, 2)
        base = cum[i]
        label_val = (f"{v:+.1f}" if is_change else f"{(base + v):.1f}")
        va = "bottom" if (base + v) >= base else "top"
        ax.text(i, base + v + (off if va == "bottom" else -off),
                label_val, ha="center", va=va, fontsize=9)

    ax.set_xticks(range(4), steps)
    ax.set_ylabel("kg CO₂e / m²")
    ax.set_title("Δ (A1–A4) decomposition — what changed?", pad=12)
    _strip_axes(ax, keep_grid=True)
    ax.figure.tight_layout(pad=0.8)

# ----------------- Category dropdown helpers (hide A4) -----------------
def _is_a4_like(name: str) -> bool:
    s = re.sub(r"[^a-z0-9]+", "", str(name).lower())
    return ("a4" in s) and ("wall" in s)

def _build_category_options(sheets: dict) -> list[tuple[str, str]]:
    """
    Return dropdown options as [(label, actual_sheet_name), ...] for:
    Wall Systems, Cladding Systems, Slab Systems. Excludes any '(A4)' sheets.
    """
    wanted = {"wall": "Wall Systems", "cladding": "Cladding Systems", "slab": "Slab Systems"}
    picked: dict[str, str] = {}
    for sname, df in sheets.items():
        if _is_a4_like(sname):  # skip the A4 sheet
            continue
        cat = _derive_system_type_key_from_df(df, sname)  # 'wall'|'cladding'|'slab'|'generic'
        if cat in wanted and wanted[cat] not in picked:
            picked[wanted[cat]] = sname
    order = ["Wall Systems", "Cladding Systems", "Slab Systems"]
    return [(label, picked[label]) for label in order if label in picked]

# ====================== App (with A4 toggle) ======================
class MultiSheetMMCApp:
    def __init__(self, sheets: dict):
        self.sheets = sheets
        self._suspend = False
        self.viable_methods_set = globals().get('VIABLE_METHODS_SET', None)

        # Build clean System Category dropdown (labels) mapped to actual sheet names (values)
        cat_options = _build_category_options(self.sheets)
        if not cat_options:
            fallback = [(s, s) for s in self.sheets.keys() if not _is_a4_like(s)]
            if not fallback:
                first = list(self.sheets.keys())[0]
                fallback = [(first, first)]
            cat_options = fallback

        self.dd_sheet = w.Dropdown(description="", options=cat_options,
                                   value=cat_options[0][1], layout=w.Layout(width="340px"))

        # MMC categories (from MMC Method)
        self.dd_mmc_a = w.Dropdown(description="", options=[], value=None, layout=w.Layout(width="300px"))
        self.dd_sys_a = w.Dropdown(description="", options=[], value=None, layout=w.Layout(width="520px"))
        self.dd_mmc_b = w.Dropdown(description="", options=[], value=None, layout=w.Layout(width="300px"))
        self.dd_sys_b = w.Dropdown(description="", options=[], value=None, layout=w.Layout(width="520px"))

        # A4 toggle (applies only when System Type = Wall Systems)
        self.chk_a4 = w.Checkbox(description="Include A4 (Walls only)", value=True, indent=False)

        # Outputs
        self.out_cards_a = w.Output(); self.out_table_a = w.Output(); self.out_charts_a = w.Output()
        self.out_cards_b = w.Output(); self.out_table_b = w.Output(); self.out_charts_b = w.Output()
        self.out_compare = w.Output()

        # Populate initial options
        self._suspend = True
        self._populate_mmc_both(); self._populate_sys_both()
        self._suspend = False

        # Events
        self.dd_sheet.observe(self._on_sheet_changed, names="value")
        self.dd_mmc_a.observe(self._on_mmc_a, names="value")
        self.dd_sys_a.observe(self._render_a, names="value")
        self.dd_mmc_b.observe(self._on_mmc_b, names="value")
        self.dd_sys_b.observe(self._render_b, names="value")
        self.chk_a4.observe(lambda _: self._render_all(), names="value")

        # Initial render
        self._render_a(None); self._render_b(None); self._render_compare()

    # ---------- viability helpers ----------
    @staticmethod
    def _norm_key(s): return str(s).strip().casefold()
    def _match_viable(self, option_label: str) -> bool:
        if not self.viable_methods_set: return True
        opt = self._norm_key(option_label)
        for m in self.viable_methods_set:
            if opt == m or opt in m or m in opt: return True
        return False
    def set_viable_methods(self, viable_set):
        self.viable_methods_set = set(viable_set) if viable_set else None
        self._suspend = True; self._populate_mmc_both(); self._populate_sys_both(); self._suspend = False
        self._render_all()

    # ---------- helpers for options ----------
    def _mmc_options(self):
        df = self.sheets[self.dd_sheet.value]
        mmc_col = _get_mmc_method_col(df)
        if not mmc_col: return ["(All)"]
        vals = []
        for v in df[mmc_col].dropna().astype(str):
            cho = _canon_choice(v, MMC_METHOD_SYNONYMS, allowed=MMC_CAT_ALLOWED)
            if cho: vals.append(cho)
        vals = sorted(set(vals))
        if vals: vals = [v for v in vals if self._match_viable(v)]
        return ["(All)"] + (vals if vals else [])

    def _set_value_safely(self, widget, value):
        if widget.value != value: widget.value = value

    def _populate_mmc_both(self):
        opts = self._mmc_options()
        self.dd_mmc_a.options = opts; self.dd_mmc_b.options = opts
        self._set_value_safely(self.dd_mmc_a, "(All)"); self._set_value_safely(self.dd_mmc_b, "(All)")

    def _populate_sys(self, which):
        df = self.sheets[self.dd_sheet.value]
        if which == 'A':
            mmc = None if self.dd_mmc_a.value in [None,"(All)"] else self.dd_mmc_a.value
            opts = systems_for_filters(df, mmc)
            self.dd_sys_a.options = opts
            if opts: self._set_value_safely(self.dd_sys_a, opts[0][1])
        else:
            mmc = None if self.dd_mmc_b.value in [None,"(All)"] else self.dd_mmc_b.value
            opts = systems_for_filters(df, mmc)
            self.dd_sys_b.options = opts
            if opts: self._set_value_safely(self.dd_sys_b, opts[0][1])

    def _populate_sys_both(self): self._populate_sys('A'); self._populate_sys('B')
    def _render_all(self): self._render_a(None); self._render_b(None); self._render_compare()

    # ---------- event handlers ----------
    def _on_sheet_changed(self, _):
        if self._suspend: return
        self._suspend = True; self._populate_mmc_both(); self._populate_sys_both(); self._suspend = False
        self._render_all()
    def _on_mmc_a(self, _):
        if self._suspend: return
        self._suspend = True; self._populate_sys('A'); self._suspend = False
        self._render_a(None)
    def _on_mmc_b(self, _):
        if self._suspend: return
        self._suspend = True; self._populate_sys('B'); self._suspend = False
        self._render_b(None)

    # ---------- renderers ----------
    def _render_a(self, _):
        if self._suspend: return
        df_sheet = self.sheets[self.dd_sheet.value]
        mmc = None if self.dd_mmc_a.value in [None,"(All)"] else self.dd_mmc_a.value
        sid = self.dd_sys_a.value
        if sid is None: return
        df_sys = slice_system(df_sheet, mmc, sid)
        sname = _get_sys_name(df_sys, f"{self.dd_sheet.value} (selection)")
        subtitle, kpis = kpis_for_sheet(self.dd_sheet.value, df_sys, include_a4=self.chk_a4.value)
        a4_hit = _lookup_a4_for_selection(df_sys, self.dd_sheet.value) if self.chk_a4.value else None
        note = None
        if _derive_system_type_key_from_df(df_sys, self.dd_sheet.value) == "wall":
            if a4_hit is None: note = "A4 not available for this wall in the template."
            else:
                parts = []
                if a4_hit.get("km") is not None and np.isfinite(a4_hit["km"]): parts.append(f"{_fmt_smart(a4_hit['km'])} km")
                if parts: note = "A4 source: " + ", ".join(parts)
        with self.out_cards_a: clear_output(wait=True); _render_kpi_cards(sname, subtitle + " — Option A", kpis, THEME_A, a4_note=note)
        with self.out_table_a: clear_output(wait=True); _styled_layers(df_sys, f"{self.dd_sheet.value} — System Layers (A)", self.dd_sheet.value)
        with self.out_charts_a: clear_output(wait=True); _charts_for(df_sys, THEME_A)
        self._render_compare()

    def _render_b(self, _):
        if self._suspend: return
        df_sheet = self.sheets[self.dd_sheet.value]
        mmc = None if self.dd_mmc_b.value in [None,"(All)"] else self.dd_mmc_b.value
        sid = self.dd_sys_b.value
        if sid is None: return
        df_sys = slice_system(df_sheet, mmc, sid)
        sname = _get_sys_name(df_sys, f"{self.dd_sheet.value} (selection)")
        subtitle, kpis = kpis_for_sheet(self.dd_sheet.value, df_sys, include_a4=self.chk_a4.value)
        a4_hit = _lookup_a4_for_selection(df_sys, self.dd_sheet.value) if self.chk_a4.value else None
        note = None
        if _derive_system_type_key_from_df(df_sys, self.dd_sheet.value) == "wall":
            if a4_hit is None: note = "A4 not available for this wall in the template."
            else:
                parts = []
                if a4_hit.get("km") is not None and np.isfinite(a4_hit["km"]): parts.append(f"{_fmt_smart(a4_hit['km'])} km")
                if parts: note = "A4 source: " + ", ".join(parts)
        with self.out_cards_b: clear_output(wait=True); _render_kpi_cards(sname, subtitle + " — Option B", kpis, THEME_B, a4_note=note)
        with self.out_table_b: clear_output(wait=True); _styled_layers(df_sys, f"{self.dd_sheet.value} — System Layers (B)", self.dd_sheet.value)
        with self.out_charts_b: clear_output(wait=True); _charts_for(df_sys, THEME_B)
        self._render_compare()

    def _comparison_html(self, sa, sb):
        def _row(label, a, b):
            try: diff = (np.nan if pd.isna(b) else b) - (np.nan if pd.isna(a) else a)
            except Exception: diff = np.nan
            return f"""
            <tr>
              <td style="padding:6px 8px;border-bottom:1px solid #f3f3f3">{label}</td>
              <td style="padding:6px 8px;text-align:right;border-bottom:1px solid #f3f3f3;color:{THEME_A['heading']}">{_fmt_smart(a)}</td>
              <td style="padding:6px 8px;text-align:right;border-bottom:1px solid #f3f3f3;color:{THEME_B['heading']}">{_fmt_smart(b)}</td>
              <td style="padding:6px 8px;text-align:right;border-bottom:1px solid #f3f3f3">{_fmt_smart(diff)}</td>
            </tr>"""
        rows = []
        if _has_finite(sa["mass"]) or _has_finite(sb["mass"]): rows.append(_row("Mass (kg/m²)", sa["mass"], sb["mass"]))
        if _has_finite(sa["a1a3"]) or _has_finite(sb["a1a3"]): rows.append(_row("A1–A3 (kg CO₂e/m²)", sa["a1a3"], sb["a1a3"]))
        if _has_finite(sa.get("a4")) or _has_finite(sb.get("a4")): rows.append(_row("A4 (transport) (kg CO₂e/m²)", sa.get("a4"), sb.get("a4")))
        if _has_finite(sa.get("a1a4")) or _has_finite(sb.get("a1a4")): rows.append(_row("A1–A4 (kg CO₂e/m²)", sa.get("a1a4"), sb.get("a1a4")))
        if _has_finite(sa.get("cf")) or _has_finite(sb.get("cf")): rows.append(_row("Avg CF (kg CO₂e/kg)", sa.get("cf"), sb.get("cf")))
        html = f"""
        <div style="display:flex;align-items:center;gap:12px;margin:10px 0 4px 0">
          <div class="mmc3-title">Comparison Report</div>
        </div>
        <table style="border-collapse:collapse;width:100%;max-width:950px;margin-bottom:6px;background:transparent">
          <thead>
            <tr>
              <th style="text-align:left;border-bottom:2px solid {THEME_A['card_border']};padding:6px 8px;background:transparent">Metric</th>
              <th style="text-align:right;border-bottom:2px solid {THEME_A['card_border']};padding:6px 8px;color:{THEME_A['heading']};background:transparent">Option A</th>
              <th style="text-align:right;border-bottom:2px solid {THEME_B['card_border']};padding:6px 8px;color:{THEME_B['heading']};background:transparent">Option B</th>
              <th style="text-align:right;border-bottom:2px solid #ddd;padding:6px 8px;background:transparent">Δ (B − A)</th>
            </tr>
          </thead>
          <tbody>
            {''.join(rows)}
          </tbody>
        </table>
        """
        return html

    def _render_compare(self):
        with self.out_compare:
            clear_output(wait=True)
            if not (self.dd_sys_a.value and self.dd_sys_b.value): return

            df_sheet = self.sheets[self.dd_sheet.value]
            df_a = slice_system(df_sheet, None if self.dd_mmc_a.value in [None,"(All)"] else self.dd_mmc_a.value, self.dd_sys_a.value)
            df_b = slice_system(df_sheet, None if self.dd_mmc_b.value in [None,"(All)"] else self.dd_mmc_b.value, self.dd_sys_b.value)

            def summary(df_sys):
                mass_m2 = _find_alias_main(df_sys, "Mass_per_m2_kg")
                a1a3_m2 = _find_alias_main(df_sys, "A1A3_kgCO2e_per_m2")
                cf_col  = _find_alias_main(df_sys, "CarbonFactor_kgCO2e_per_kg")
                name = _get_sys_name(df_sys, self.dd_sheet.value)
                mass = _sum(df_sys, mass_m2) if mass_m2 else np.nan
                a1a3 = _sum(df_sys, a1a3_m2) if a1a3_m2 else np.nan
                cf   = _mass_weighted_cf(df_sys) if cf_col else np.nan
                a4hit = _lookup_a4_for_selection(df_sys, self.dd_sheet.value) if self.chk_a4.value else None
                a4 = (a4hit.get("A4") if a4hit is not None else np.nan)
                a1a4 = (a1a3 + a4) if (_has_finite(a1a3) and _has_finite(a4)) else np.nan
                return dict(name=name, mass=mass, a1a3=a1a3, a4=a4, a1a4=a1a4, cf=cf)

            sa, sb = summary(df_a), summary(df_b)
            ipy_display(HTML(self._comparison_html(sa, sb)))

            left = w.Output(); right = w.Output()
            ipy_display(w.HBox([left, right], layout=w.Layout(justify_content="space-between", width="100%")))

            use_a4_for_y = self.chk_a4.value and _has_finite(sa.get("a1a4")) and _has_finite(sb.get("a1a4"))

            with left:
                condA = _has_finite(sa["mass"]) and (_has_finite(sa["a1a3"]) or _has_finite(sa.get("a1a4")))
                condB = _has_finite(sb["mass"]) and (_has_finite(sb["a1a3"]) or _has_finite(sb.get("a1a4")))
                if condA and condB:
                    fig, ax = plt.subplots(figsize=(6.8, 3.6))
                    _plot_tradeoff_mass_vs_carbon(ax, sa, sb, use_a4_for_y, THEME_A["bar"], THEME_B["bar"])
                    plt.show()

            with right:
                if use_a4_for_y:
                    fig2, ax2 = plt.subplots(figsize=(6.8, 3.6))
                    _plot_waterfall_a1a4(ax2, sa, sb, color_pos=THEME_B["bar"], color_neg="#F28482")
                    plt.show()
                else:
                    if _has_finite(sa["a1a3"]) and _has_finite(sb["a1a3"]) and _has_finite(sa["mass"]) and _has_finite(sb["mass"]):
                        gwp_a, gwp_b = float(sa["a1a3"]), float(sb["a1a3"])
                        mass_a, mass_b = float(sa["mass"]), float(sb["mass"])
                        d_gwp = (gwp_b - gwp_a) / max(1e-9, gwp_a) * 100.0
                        d_mass = (mass_b - mass_a) / max(1e-9, mass_a) * 100.0
                        fig3, ax3 = plt.subplots(figsize=(6.8, 3.6))
                        labels = ["GWP (A1–A3)", "Mass"]; diffs = [d_gwp, d_mass]
                        colors = [("#9AD0C2" if v <= 0 else "#F28482") for v in diffs]
                        ax3.barh(labels, diffs, color=colors)
                        ax3.axvline(0, color="#999", lw=1)
                        ax3.set_xlabel("Change B vs A (%) — left = reduction, right = increase")
                        ax3.set_title("Improvement summary", pad=10)
                        for i, v in enumerate(diffs):
                            ax3.text(v + (1.2 if v >= 0 else -1.2), i, f"{v:+.1f}%", va="center",
                                     ha=("left" if v >= 0 else "right"))
                        _strip_axes(ax3, keep_grid=True)
                        ax3.figure.tight_layout(pad=0.8)
                        plt.show()

    # ---------- layout ----------
    def show(self):
        # Global styles; targeted slider removal for MMC containers + transparent tables
        css = f"""
        <style>
          :root {{
            --accA:{THEME_A['accent']}; --accB:{THEME_B['accent']};
            --card-bdrA:{THEME_A['card_border']}; --card-bdrB:{THEME_B['card_border']};
            --card-bgA:{THEME_A['card_bg']}; --card-bgB:{THEME_B['card_bg']};
            --headingA:{THEME_A['heading']}; --headingB:{THEME_B['heading']};
          }}
          .mmc3-app {{
            background: var(--card-bgA);
            border: 1px solid var(--card-bdrA);
            border-radius: 18px;
            padding: 12px 14px;
            box-shadow: 0 10px 32px rgba(16,24,40,.05) inset;
            overflow-x: hidden;
            font-family: ui-sans-serif, system-ui, -apple-system, Segoe UI, Roboto, Helvetica, Arial;
          }}
          .mmc3-title {{ font-weight:800; color:{THEME_A['heading']}; font-size:16px; letter-spacing:.2px; }}
          .mmc3-subtle {{ color:#64748b; font-size:12px; }}
          .mmc3-hr {{ border:0; border-top:1px dashed var(--card-bdrA); margin:10px 0; }}
          .mmc3-label {{ font-weight:700; font-size:12px; color:#1f2937; margin:2px 0 6px; }}

          /* Dropdown polish */
          .mmc3-app .widget-dropdown select {{
            border:1px solid var(--card-bdrA) !important;
            background:transparent !important;
            border-radius:12px !important;
            padding:6px 8px !important;
            box-shadow: 0 1px 2px rgba(16,24,40,.06) inset;
          }}
          
          /* -------- Remove sliders (global & under MMC blocks) -------- */
          .mmc3-app .widget-slider,
          .mmc3-app .widget-intslider,
          .mmc3-app .widget-floatslider,
          .mmc3-app .widget-rangeslider,
          .mmc3-app .widget-selectionslider,
          .mmc3-app .widget-selectionrangeslider,
          .mmc3-app input[type="range"] {{
            display:none !important; visibility:hidden !important;
            width:0 !important; height:0 !important; margin:0 !important; padding:0 !important; border:0 !important;
          }}
          .mmc3-app .mmc3-mmc [class*="slider"] {{
            display:none !important; visibility:hidden !important;
            width:0 !important; height:0 !important; margin:0 !important; padding:0 !important; border:0 !important;
          }}

          /* Checkbox accent */
          .mmc3-app .widget-checkbox input {{ accent-color: var(--accA) !important; }}

          /* ToggleButtons — persistent highlight */
          .mmc3-app .widget-toggle-buttons .widget-toggle-button.mod-active,
          .mmc3-app .widget-toggle-buttons .widget-toggle-button[aria-pressed="true"],
          .mmc3-app .widget-toggle-buttons .widget-toggle-button[aria-checked="true"] {{
            background: var(--accA) !important; color:#fff !important; border-color: var(--accA) !important;
            box-shadow: 0 0 0 2px rgba(116,198,157,.25) inset;
          }}
          .mmc3-app .widget-toggle-buttons .widget-toggle-button {{
            border:1px solid var(--card-bdrA); background:#fff; color:#0f172a;
            border-radius:999px !important; margin:2px; box-shadow:0 1px 2px rgba(16,24,40,.06) inset;
          }}

          /* Transparent pandas Styler tables (override inline) */
          .mmc3-app table.dataframe,
          .mmc3-app table.dataframe thead tr,
          .mmc3-app table.dataframe tbody tr,
          .mmc3-app table.dataframe th,
          .mmc3-app table.dataframe td {{ background: transparent !important; }}
        </style>
        """
        ipy_display(HTML(css))

        # Labels
        lbl_sheet = w.HTML("<div class='mmc3-label'>System Category</div>")
        lbl_mmcA  = w.HTML("<div class='mmc3-label'>MMC Category</div>")
        lbl_sysA  = w.HTML("<div class='mmc3-label'>System Selection</div>")
        lbl_mmcB  = w.HTML("<div class='mmc3-label'>MMC Category</div>")
        lbl_sysB  = w.HTML("<div class='mmc3-label'>System Selection</div>")

        # Header
        header = w.HBox([
            w.VBox([w.HTML("<div class='mmc3-title'>MMC Generic Model Assessment</div>"),
                    w.HTML("<div class='mmc3-subtle'>Compare KPIs of generic systems (A/B), normalized categories, optional A4.</div>")],
                   layout=w.Layout(flex='1 1 auto')),
            w.HBox([self.chk_a4], layout=w.Layout(align_items='center'))
        ], layout=w.Layout(width='100%', align_items='center'))

        # Category (sheet) selector
        sheet_row = w.GridBox(
            children=[w.VBox([lbl_sheet, self.dd_sheet])],
            layout=w.Layout(grid_template_columns="minmax(360px, 360px)", width='100%')
        )
        hr1 = w.HTML("<hr class='mmc3-hr'>")

        # Option A block (mark MMC column with class to target sliders)
        mmcA_box = w.VBox([lbl_mmcA, self.dd_mmc_a], _dom_classes=['mmc3-mmc'])
        sysA_box = w.VBox([lbl_sysA, self.dd_sys_a])
        rowA = w.GridBox(
            children=[mmcA_box, sysA_box],
            layout=w.Layout(grid_template_columns="minmax(360px, 360px) minmax(320px, 1fr)", grid_gap="20px", width='100%')
        )
        blockA = w.VBox([
            w.HTML("<div class='mmc3-title' style='margin-top:20px'>Option A</div>"),
            rowA,
            self.out_cards_a, self.out_table_a, self.out_charts_a
        ], layout=w.Layout(width='100%'))

        hr2 = w.HTML("<hr class='mmc3-hr'>")

        # Option B block
        mmcB_box = w.VBox([lbl_mmcB, self.dd_mmc_b], _dom_classes=['mmc3-mmc'])
        sysB_box = w.VBox([lbl_sysB, self.dd_sys_b])
        rowB = w.GridBox(
            children=[mmcB_box, sysB_box],
            layout=w.Layout(grid_template_columns="minmax(360px, 360px) minmax(320px, 1fr)", grid_gap="20px", width='100%')
        )
        blockB = w.VBox([
            w.HTML("<div class='mmc3-title' style='margin-top:20px'>Option B</div>"),
            rowB,
            self.out_cards_b, self.out_table_b, self.out_charts_b
        ], layout=w.Layout(width='100%'))

        hr3 = w.HTML("<hr class='mmc3-hr'>")

        # Comparison
        compare_block = w.VBox([self.out_compare], layout=w.Layout(width='100%'))

        shell = w.VBox([header, sheet_row, hr1, blockA, hr2, blockB, hr3, compare_block],
                       layout=w.Layout(width='100%'),
                       _dom_classes=['mmc3-app'])
        ipy_display(shell)


### MMC Generic Model Assessment UI

In [22]:
# -------------------- Launch --------------------
app = MultiSheetMMCApp(SHEETS)
app.show()

##  Ireland Manufacturer Map

### Backend

In [23]:
# ========================= Section 3 — Ireland map (white-board/card style) =========================
# Requires:
#   pip install folium
#   (If your file is .xls) pip install xlrd
#   (Optional for auto coordinate fix) pip install pyproj

from pathlib import Path
import math, json, hashlib, unicodedata, re
import numpy as np
import pandas as pd
import ipywidgets as w
from IPython.display import HTML, display, clear_output

# ---------- Canonical MMC list ----------
ALLOWED_MMC = [
    "Concrete Block",
    "ICF (Insulating Concrete Formwork)",
    "LGS (Light Gauge Steel)",
    "Timber",
]

def _canon_mmc(val: str) -> str | None:
    s = (str(val) or "").strip().lower()
    if not s: return None
    if "concrete" in s and "block" in s: return "Concrete Block"
    if s == "icf" or "insulating concrete form" in s or s.startswith("icf "): return "ICF (Insulating Concrete Formwork)"
    if s == "lgs" or "light gauge steel" in s or "light-gauge steel" in s:   return "LGS (Light Gauge Steel)"
    if "timber" in s or "wood" in s:                                         return "Timber"
    return None

# ---------- 1) Load manufacturers ----------
def _load_manufacturers():
    candidates = [
        Path("mmc_manufacturer_map_list.xlsx"),
        Path("mmc_manufacturer_map_list.xls"),
        Path("mmc_manufacturer_map_list.csv"),
    ]
    p = next((pp for pp in candidates if pp.exists()), None)
    if p is None:
        raise FileNotFoundError("Manufacturer list not found (xlsx/xls/csv).")

    ext = p.suffix.lower()
    if ext == ".xlsx":
        df = pd.read_excel(p, engine="openpyxl")
    elif ext == ".xls":
        try:
            import xlrd  # noqa: F401
            df = pd.read_excel(p, engine="xlrd")
        except Exception as e:
            raise ImportError("The file is .xls but 'xlrd' is not installed. Install xlrd or save as .xlsx/.csv.") from e
    elif ext == ".csv":
        df = pd.read_csv(p)
    else:
        df = pd.read_excel(p)

    def _norm_key(s): 
        return re.sub(r"[^a-z0-9]+", "_", str(s).strip().lower())

    cols = {_norm_key(c): c for c in df.columns}

    def get_col(*cands):
        for c in cands:
            c_norm = _norm_key(c)
            if c_norm in cols: return cols[c_norm]
        return None

    name_col   = get_col('manufacturer', 'manufacturer_name', 'name', 'company', 'organisation')
    cat_col    = get_col('mmc_method', 'mmc category', 'mmc_category', 'mmc', 'category')
    county_col = get_col('county', 'county_name')
    x_col      = get_col('point_x', 'lon', 'longitude', 'x')
    y_col      = get_col('point_y', 'lat', 'latitude', 'y')
    url_col    = get_col('website', 'url', 'link')

    needed = [name_col, cat_col, county_col, x_col, y_col]
    if any(c is None for c in needed):
        raise RuntimeError(
            "Manufacturer sheet must contain columns for manufacturer, MMC category/method, county, point_x, point_y "
            "(plus optional website)."
        )

    out = pd.DataFrame(dict(
        Manufacturer = df[name_col].astype(str).str.strip(),
        MMC_Category = df[cat_col].map(_canon_mmc),
        County       = df[county_col].astype(str).str.strip(),
        X            = pd.to_numeric(df[x_col], errors="coerce"),
        Y            = pd.to_numeric(df[y_col], errors="coerce"),
        Website      = (df[url_col].astype(str).str.strip() if url_col else "")
    ))
    out = out.dropna(subset=["X","Y","MMC_Category"]).reset_index(drop=True)
    return out

# ---------- 2) Coordinate fixer -> WGS84 ----------
def _to_wgs84(df_xy):
    x = df_xy["X"].astype(float).values
    y = df_xy["Y"].astype(float).values

    LON_MIN, LON_MAX = -11.2, -5.2
    LAT_MIN, LAT_MAX =  51.0,  55.7

    def score(lon, lat):
        ok_geo = (np.isfinite(lon) & np.isfinite(lat) &
                  (np.abs(lon) <= 180) & (np.abs(lat) <= 90))
        in_ie  = ok_geo & (lon >= LON_MIN) & (lon <= LON_MAX) & (lat >= LAT_MIN) & (lat <= LAT_MAX)
        return ok_geo.mean(), in_ie.sum()

    s_geo1, s_ie1 = score(x, y)
    s_geo2, s_ie2 = score(y, x)

    note = ""
    lon, lat = x, y
    if (s_ie2 > s_ie1) or ((s_ie1 == 0) and (s_geo2 > s_geo1)):
        lon, lat = y, x
        note = "Auto-fixed swapped lat/lon"

    if not ((np.abs(lon) <= 180).all() and (np.abs(lat) <= 90).all()):
        try:
            from pyproj import Transformer
            for epsg in (2157, 29903, 3857):  # ITM, Irish Grid, Web-Mercator
                tr = Transformer.from_crs(epsg, 4326, always_xy=True)
                LON, LAT = tr.transform(lon, lat)
                _, in_ie = score(np.array(LON), np.array(LAT))
                if in_ie > 0:
                    lon, lat = np.array(LON), np.array(LAT)
                    note = f"Transformed from EPSG:{epsg}"
                    break
        except Exception:
            R = 6378137.0
            if np.nanmax(np.abs(lon)) > 1e5 and np.nanmax(np.abs(lat)) > 1e5:
                LON = (lon / R) * 180.0 / math.pi
                LAT = (2 * np.arctan(np.exp(lat / R)) - math.pi/2) * 180.0 / math.pi
                lon, lat = LON, LAT
                note = "Approximated from Web-Mercator"

    out = df_xy.copy()
    out["Lon"], out["Lat"] = lon, lat
    return out, note

# ---------- 3) Marker colors ----------
MARKER_EXPLICIT = {
    "Concrete Block": "#2563EB",
    "ICF (Insulating Concrete Formwork)": "#D97706",
    "LGS (Light Gauge Steel)": "#0D9488",
    "Timber": "#7C3AED",
}
MARKER_FALLBACK = [
    "#2563EB", "#0EA5E9", "#1D4ED8", "#7C3AED", "#A855F7",
    "#D97706", "#F97316", "#E11D48", "#334155", "#06B6D4",
]
def _stable_marker_color(name: str) -> str:
    if not name: return MARKER_FALLBACK[0]
    h = int(hashlib.sha1(name.encode("utf-8")).hexdigest()[:8], 16)
    return MARKER_FALLBACK[h % len(MARKER_FALLBACK)]

def _build_marker_color_map(categories):
    cmap = {}
    for cat in categories:
        cat_s = str(cat).strip()
        cmap[cat_s] = MARKER_EXPLICIT.get(cat_s, _stable_marker_color(cat_s))
    return cmap

def _clean_url(u: str) -> str:
    if not u: return ""
    u = u.strip()
    if u and not u.lower().startswith(("http://","https://")):
        return "https://" + u
    return u

# ---------- 4) County polygons ----------
def _load_counties_geojson():
    candidates = [Path("ireland_counties.json"), Path("ireland_counties.geojson")]
    p = next((pp for pp in candidates if pp.exists()), None)
    if not p: return None, "County polygons not found — shading disabled. Add 'ireland_counties*.geojson'."
    try:
        with open(p, "r", encoding="utf-8") as f:
            gj = json.load(f)
        return gj, f"County polygons: {p.name}"
    except Exception as e:
        return None, f"Failed to read county polygons: {e}"

# ---------- 5) Name normalisation ----------
_alias_map = {"countydublin":"dublin","codublin":"dublin","countycork":"cork","cocork":"cork"}
def _strip_diacritics(s: str) -> str:
    return unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
def _norm_county_name(s: str) -> str:
    s = _strip_diacritics(str(s))
    s = s.strip().lower()
    s = re.sub(r"^(co\.?|county)\s+", "", s)
    s = re.sub(r"\b(city|county|council)\b", "", s)
    s = re.sub(r"[^a-z0-9]+", "", s)
    return _alias_map.get(s, s)

# ---------- 6) Map class (white-board/card UI) ----------
class ManufacturerMapApp:
    IRE_TIGHT_BOUNDS = [[51.35, -7.80], [55.45, -5.38]]
    START_ZOOM = 7
    MAX_ZOOM   = 10
    SHIFT_LON_DEG = 1.00   # push Ireland left in viewport
    SHIFT_LAT_DEG = 0.00

    def __init__(self, df: pd.DataFrame):
        self.raw = df.copy()
        self.geo, self._coord_note = _to_wgs84(self.raw)
        self.viable_set    = globals().get("VIABLE_METHODS_SET", None)
        self.county_geojson, self._county_note = _load_counties_geojson()

        # ---------- UI (white-board style) ----------
        css = f"""
        <style>
          .mmc-map {{
            background: {CARD_BG};
            border: 1px solid {CARD_BORDER};
            border-radius: 18px;
            padding: 12px 14px;
            box-shadow: 0 1px 2px rgba(15,23,42,.06), 0 8px 32px rgba(15,23,42,.08);
            overflow: hidden;
            font-family: ui-sans-serif, system-ui, -apple-system, Segoe UI, Roboto, Helvetica, Arial;
          }}
          .mmc-map .title {{ font-weight:800; color:{HEADING}; font-size:16px; letter-spacing:.2px; }}
          .mmc-map .subtle {{ color:#64748b; font-size:12px; }}
          .mmc-map .chip {{
            background:#fff; border:1px solid {CARD_BORDER};
            padding:4px 10px;border-radius:999px;font-weight:700;color:{HEADING};font-size:12px;
            box-shadow: 0 1px 2px rgba(0,0,0,.04);
          }}
          .mmc-map input[type="checkbox"] {{ accent-color: {ACCENT}; }}
        </style>
        """
        display(HTML(css))

        self.status_chip   = w.HTML()
        self.note_chip     = w.HTML(f"<span class='subtle'>{self._coord_note or ''}</span>")
        self.note_chip2    = w.HTML(f"<span class='subtle'>{self._county_note or ''}</span>")
        self.shade_toggle  = w.Checkbox(value=True, description="Shade counties by # manufacturers", indent=False)
        self.map_out       = w.Output(layout=w.Layout(width="100%"))

        title = w.HTML(
            "<div class='title'>Ireland — MMC Manufacturer Map</div>"
            "<div class='subtle' style='margin-top:2px'>Circles = manufacturers (distinct palette). "
            "County shading (optional) = green gradient by manufacturer count. "
            "Zoom-in only (no zoom-out). Click a label to open the website.</div>"
        )

        header = w.VBox([
            title,
            w.HBox([self.shade_toggle, w.HTML("<div style='flex:1'></div>"), self.status_chip],
                   layout=w.Layout(align_items="center")),
            self.note_chip, self.note_chip2
        ])

        self.card = w.VBox([header, self.map_out],
                           layout=w.Layout(width="100%"),
                           _dom_classes=['mmc-map','wb-skin'])

        self.shade_toggle.observe(lambda *_: self._render(), names="value")
        self._render()

    def show(self): display(self.card)

    def set_viable_methods(self, viable_set):
        self.viable_set = set(viable_set) if viable_set else None
        self._render()

    @staticmethod
    def _match_viable(cat: str, viable_set) -> bool:
        if not viable_set: return True
        key = str(cat).strip().casefold()
        for m in viable_set:
            mm = str(m).strip().casefold()
            if key == mm or key in mm or mm in key:
                return True
        return False

    def _render(self):
        import folium
        from folium import Map, TileLayer, FeatureGroup, Popup, IFrame, Circle, CircleMarker, LayerControl, GeoJson
        from folium.features import DivIcon
        from branca.colormap import LinearColormap

        with self.map_out:
            clear_output(wait=True)

            # Filter: Ireland bbox + viability
            df = self.geo.copy()
            bbox = (df["Lon"].between(-11.2, -5.2) & df["Lat"].between(51.0, 55.7))
            df = df.loc[bbox].copy()
            df = df[df["MMC_Category"].apply(lambda c: self._match_viable(c, self.viable_set))].copy()

            n = len(df); n_counties = df["County"].nunique()
            self.status_chip.value = f"<span class='chip'>{n} manufacturers across {n_counties} counties</span>"

            if df.empty:
                display(HTML("<div style='margin:8px 0;color:#b42318;'>No manufacturers to show for current filter.</div>"))
                return

            # Colors
            cats = sorted(df["MMC_Category"].dropna().astype(str).unique())
            color_map = _build_marker_color_map(cats)

            # ---------- Basemaps ----------
            BASEMAPS = [
                ("Light (Positron)", "CartoDB positron", None, True),
                ("Dark (Carto)",    "CartoDB dark_matter", None, False),
                ("Muted Gray (Esri)", "https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}",
                 "Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ", False),
                ("OSM", "OpenStreetMap", None, False),
            ]
            center = [(self.IRE_TIGHT_BOUNDS[0][0] + self.IRE_TIGHT_BOUNDS[1][0]) / 2.0,
                      (self.IRE_TIGHT_BOUNDS[0][1] + self.IRE_TIGHT_BOUNDS[1][1]) / 2.0]
            m = Map(location=center, zoom_start=self.START_ZOOM, tiles=None,
                    control_scale=True, prefer_canvas=True, zoom_control=True)

            for name, tiles, attr, show in BASEMAPS:
                if tiles in ("CartoDB positron", "CartoDB dark_matter", "OpenStreetMap"):
                    TileLayer(tiles, name=name, show=show).add_to(m)
                else:
                    TileLayer(tiles=tiles, name=name, attr=attr or "", show=show, max_zoom=19).add_to(m)

            # --- Fixed bounds + zoom-in only + LEFT bias + RESET button ---
            map_id = m.get_name()
            shift_and_reset_js = f"""
            <script>
              (function() {{
                var map = {map_id};
                var bounds = L.latLngBounds(
                  [{self.IRE_TIGHT_BOUNDS[0][0]}, {self.IRE_TIGHT_BOUNDS[0][1]}],
                  [{self.IRE_TIGHT_BOUNDS[1][0]}, {self.IRE_TIGHT_BOUNDS[1][1]}]
                );

                var START_ZOOM = {self.START_ZOOM};
                var MAX_ZOOM   = {self.MAX_ZOOM};
                var SHIFT_LON_DEG = {self.SHIFT_LON_DEG};
                var SHIFT_LAT_DEG = {self.SHIFT_LAT_DEG};

                function getShiftedCenter() {{
                  var lat = ({self.IRE_TIGHT_BOUNDS[0][0]} + {self.IRE_TIGHT_BOUNDS[1][0]})/2 + SHIFT_LAT_DEG;
                  var lon = ({self.IRE_TIGHT_BOUNDS[0][1]} + {self.IRE_TIGHT_BOUNDS[1][1]})/2 + SHIFT_LON_DEG;
                  return [lat, lon];
                }}

                function applyView() {{
                  map.setView(getShiftedCenter(), START_ZOOM, {{animate:false}});
                  map.setMinZoom(START_ZOOM);
                  map.setMaxZoom(MAX_ZOOM);
                  map.setMaxBounds(bounds.pad(0.03));
                  map.scrollWheelZoom.enable();
                  map.doubleClickZoom.enable();
                  map.touchZoom.enable();
                  map.boxZoom.enable();
                  map.keyboard.enable();
                  if (map.zoomControl && map.zoomControl._zoomOutButton) {{
                    map.zoomControl._zoomOutButton.style.display = 'none';
                  }}
                }}

                var ResetCtl = L.Control.extend({{
                  options: {{ position: 'topleft' }},
                  onAdd: function(map) {{
                    var btn = L.DomUtil.create('button', 'leaflet-bar reset-view-btn');
                    btn.innerHTML = '⟲';
                    btn.title = 'Reset view';
                    btn.style.width='28px'; btn.style.height='28px';
                    btn.style.lineHeight='26px'; btn.style.fontSize='16px';
                    btn.style.background='{CARD_BG}';
                    btn.style.border='1px solid {CARD_BORDER}';
                    btn.style.borderRadius='4px';
                    btn.style.boxShadow='0 1px 2px rgba(0,0,0,.06)';
                    L.DomEvent.on(btn, 'click', function(e){{
                      L.DomEvent.stopPropagation(e); L.DomEvent.preventDefault(e);
                      map.setView(getShiftedCenter(), START_ZOOM, {{animate:false}});
                    }});
                    return btn;
                  }}
                }});
                map.addControl(new ResetCtl());

                map.whenReady(applyView);
                map.on('resize baselayerchange', applyView);
                setTimeout(applyView, 60);
              }})();
            </script>
            """
            m.get_root().html.add_child(folium.Element(shift_and_reset_js))

            # ---------- County shading ----------
            shade_on = self.county_geojson is not None and self.shade_toggle.value
            if self.county_geojson is not None and shade_on:
                counts_raw = df.groupby("County").size().to_dict()
                norm_counts = { _norm_county_name(k): int(v) for k, v in counts_raw.items() }
                grp_dom = df.groupby(["County", "MMC_Category"]).size().unstack(fill_value=0)
                dominant = grp_dom.idxmax(axis=1).to_dict()
                norm_dom = { _norm_county_name(k): str(v) for k, v in dominant.items() }
                name_keys = ["name","NAME_ENGL","NAME","CountyName","COUNTYNAME","COUNTY","county","ENNAME","english","LAU_NAME"]

                gj = self.county_geojson
                vmax = max([0] + list(norm_counts.values())) or 1
                vmin = 0
                palette = ['#E9F7EF','#CDEEDB','#A5E1C9','#7FD3B9','#57C0A8','#2CA58D','#1B7F6B','#145A4A','#0F3D32']
                cmap = LinearColormap(palette, vmin=vmin, vmax=vmax)

                for feat in gj.get("features", []):
                    props = (feat.get("properties") or {}).copy()
                    label = None
                    for k in name_keys:
                        if k in props and props[k]:
                            label = str(props[k]).strip(); break
                    if not label: label = str(props.get("NAME","County")).strip()
                    key = _norm_county_name(label)
                    props["County_label"] = label
                    props["MFG_COUNT"]    = int(norm_counts.get(key, 0))
                    props["DOM_MMC"]      = norm_dom.get(key, "-")
                    feat["properties"]    = props

                def style_fn(feature):
                    c = int(feature["properties"].get("MFG_COUNT", 0))
                    return {"fillColor": cmap(c), "color":"#4B6B5A", "weight":1, "fillOpacity":0.6}

                tooltip = folium.features.GeoJsonTooltip(
                    fields=["County_label","MFG_COUNT","DOM_MMC"],
                    aliases=["County","Manufacturers","Dominant MMC"],
                    localize=True, sticky=False, labels=True
                )
                GeoJson(gj, name="County shading (by # manufacturers)",
                        style_function=style_fn, tooltip=tooltip).add_to(m)

                gradient_bar = "background: linear-gradient(to right, " + ", ".join(palette) + "); height:10px;border-radius:6px;margin-top:6px"
                county_legend = (
                    "<div style='position: fixed; top: 10px; left: 50px; z-index: 9998;"
                    f"background:{CARD_BG}; border:1px solid {CARD_BORDER}; border-radius:12px;"
                    "padding:8px 10px; box-shadow:2px 2px 4px rgba(0,0,0,0.06);'>"
                    f"<div style='font-weight:700;color:{HEADING};font-size:12px;margin-bottom:4px'>County shading</div>"
                    f"<div style='{gradient_bar}'></div>"
                    f"<div style='font-size:11px;color:#445;margin-top:4px'>{vmin}&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{vmax}</div>"
                    "</div>"
                )
                m.get_root().html.add_child(folium.Element(county_legend))

            # ---------- County bubbles (when shading off) ----------
            if not shade_on:
                county_grp = FeatureGroup(name="County overview (bubbles)", show=True).add_to(m)
                grp_b = df.groupby(["County", "MMC_Category"]).size().unstack(fill_value=0)
                totals_b = grp_b.sum(axis=1)
                dominant_b = grp_b.idxmax(axis=1)
                centroids = df.groupby("County")[["Lat","Lon"]].mean()
                for county in totals_b.index:
                    ctot = int(totals_b.loc[county]); 
                    if ctot <= 0: continue
                    dom  = str(dominant_b.loc[county])
                    lat, lon = float(centroids.loc[county,"Lat"]), float(centroids.loc[county,"Lon"])
                    hue = _build_marker_color_map([dom])[dom]
                    r = 7000 + (3500 * (np.sqrt(ctot)))
                    Circle([lat,lon], radius=r, color=hue, weight=2, opacity=0.35,
                           fill=True, fill_color=hue, fill_opacity=0.13).add_to(county_grp)

            # ---------- Manufacturer markers + translucent clickable labels ----------
            manu_grp   = FeatureGroup(name="Manufacturers", show=True).add_to(m)
            labels_grp = FeatureGroup(name="Labels (click to open)", show=True).add_to(m)

            label_css = """
            <style>
              .leaflet-div-icon.clickable-label { background: transparent; border: none; pointer-events: auto !important; }
              .mfg-pill {
                display:inline-block; max-width:240px; padding:2px 8px; border-radius:999px;
                background: rgba(255,255,255,.78); border:1px solid rgba(0,0,0,.08);
                font-size:11px; color:#111827; white-space:nowrap; overflow:hidden; text-overflow:ellipsis;
                box-shadow:0 1px 2px rgba(0,0,0,.06);
              }
              .mfg-pill a { color:#111827; text-decoration:none; font-weight:700; }
              .mfg-pill .dot { display:inline-block; width:8px; height:8px; border-radius:50%;
                                margin-right:6px; vertical-align:middle; }
            </style>
            """
            m.get_root().html.add_child(folium.Element(label_css))

            for _, r in df.iterrows():
                lat, lon = float(r["Lat"]), float(r["Lon"])
                name     = str(r["Manufacturer"])
                county   = str(r["County"])
                cat      = str(r["MMC_Category"])
                url      = _clean_url(str(r.get("Website", "") or ""))
                col      = color_map.get(cat, list(MARKER_FALLBACK)[0])

                link_html = (f"<a href='{url}' target='_blank' rel='noopener' "
                             f"style='color:{HEADING};text-decoration:none;font-weight:800'>{name} ↗</a>"
                             if url else f"<span style='font-weight:800;color:{HEADING}'>{name}</span>")
                pop_html = (f"{link_html}"
                            f"<div style='font-size:12px;color:#445;margin-top:4px'>"
                            f"MMC: <b>{cat}</b><br>County: <b>{county}</b></div>")
                popup = folium.Popup(folium.IFrame(html=pop_html, width=260, height=90), max_width=260)

                # halo + core
                CircleMarker([lat, lon], radius=10.5, weight=0,
                             color=col, fill=True, fill_color=col, fill_opacity=0.22).add_to(manu_grp)
                CircleMarker([lat, lon], radius=6, weight=2,
                             color="#ffffff", fill=True, fill_color=col, fill_opacity=0.96,
                             tooltip=f"{name} · {cat}", popup=popup).add_to(manu_grp)

                # translucent clickable label
                label_inner = (f"<div class='mfg-pill'><span class='dot' style='background:{col}'></span>"
                               f"<a href='{url}' target='_blank' rel='noopener' title='{name}'>{name}</a></div>"
                               if url else
                               f"<div class='mfg-pill'><span class='dot' style='background:{col}'></span>{name}</div>")
                folium.Marker(
                    [lat, lon],
                    icon=DivIcon(
                        html=f"<div class='clickable-label' style='transform:translate(10px,-14px);'>{label_inner}</div>",
                        icon_size=(250, 22), icon_anchor=(0, 0),
                    )
                ).add_to(labels_grp)

            # ---------- Legend ----------
            counts_cat = df.groupby("MMC_Category").size().to_dict()
            chips = [
                f"<span style='display:inline-block;padding:4px 8px;border-radius:999px;"
                f"background:{_build_marker_color_map([c])[c]};color:#fff;font-size:11px;margin:2px 4px 0 0'>{c} "
                f"<span style='opacity:.9'>({counts_cat.get(c,0)})</span></span>"
                for c in cats
            ]
            mmc_legend = (
                "<div style='position: fixed; bottom: 20px; right: 14px; z-index: 10000;"
                f"background:{CARD_BG}; border:1px solid {CARD_BORDER}; border-radius:12px;"
                "padding:8px 10px; box-shadow:2px 2px 4px rgba(0,0,0,0.06);'>"
                f"<div style='font-weight:700;color:{HEADING};font-size:12px;margin-bottom:4px'>MMC categories</div>"
                + "".join(chips) + "</div>"
            )
            m.get_root().html.add_child(folium.Element(mmc_legend))

            LayerControl(collapsed=False).add_to(m)

            root = m.get_root()
            root.width = "100%"
            root.height = "620px"
            display(m)

### Ireland Manufacturer Map UI

In [24]:
# ---- Load & launch
_mfg_df = _load_manufacturers()
mfg_map = ManufacturerMapApp(_mfg_df)
mfg_map.show()

# ---- Link to Section 1: refresh after “Apply Viability”
def refresh_mfg_map_from_viability():
    if 'mfg_map' in globals() and 'VIABLE_METHODS_SET' in globals():
        mfg_map.set_viable_methods(VIABLE_METHODS_SET)

if 'apply_btn' in globals() and not globals().get('_MAP_REFRESH_LINKED', False):
    apply_btn.on_click(lambda _: refresh_mfg_map_from_viability())
    globals()['_MAP_REFRESH_LINKED'] = True

try:
    if 'VIABLE_METHODS_SET' in globals() and VIABLE_METHODS_SET:
        refresh_mfg_map_from_viability()
except Exception:
    pass

#  Complete System Builder → Project Impact

### Backend

In [25]:
# ========================= Section 4 — Complete System Selector → Project Impact (white-board style) =========================
import os, re
from pathlib import Path
import numpy as np
import pandas as pd
import ipywidgets as w
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from IPython.display import display as ipy_display, HTML, clear_output

# ---------- Theme fallbacks (use global tokens if already defined) ----------
ACCENT      = globals().get("ACCENT",      "#74C69D")
HEADING     = globals().get("HEADING",     "#0F172A")
CARD_BG     = globals().get("CARD_BG",     "#FFFFFF")
CARD_BORDER = globals().get("CARD_BORDER", "rgba(15, 23, 42, 0.10)")

# Matplotlib: transparent backgrounds to sit on the white-board card
plt.rcParams["figure.facecolor"]  = "none"
plt.rcParams["axes.facecolor"]    = "none"
plt.rcParams["savefig.facecolor"] = "none"

# ---------- Global UI CSS (match Cells 1–3) ----------
ipy_display(HTML(f"""
<style>
  :root {{
    --acc: {ACCENT};
    --heading: {HEADING};
    --card-bg: {CARD_BG};
    --card-bdr: {CARD_BORDER};
    --ink:#0f172a; --muted:#64748b;
  }}

  /* Outer app card */
  .mmc4-app {{
    background: var(--card-bg);
    border: 1px solid var(--card-bdr);
    border-radius: 18px;
    padding: 12px 14px;
    box-shadow: 0 8px 30px rgba(16,24,40,.08);
    overflow: hidden;
    font-family: ui-sans-serif, system-ui, -apple-system, Segoe UI, Roboto, Helvetica, Arial;
  }}

  .mmc4-title   {{ font-weight:800; color:var(--heading); font-size:16px; letter-spacing:.2px; }}
  .mmc4-subtle  {{ color:var(--muted); font-size:12px; }}
  .mmc4-hr      {{ border:0; border-top:1px dashed var(--card-bdr); margin:10px 0; }}
  .mmc4-label   {{ font-weight:700; font-size:12px; color:#1f2937; margin:2px 0 6px; }}
  .mmc4-chip    {{
    display:inline-flex; gap:10px; align-items:center;
    background:#fff; color:var(--heading); font-weight:700; font-size:12px;
    border:1px solid var(--card-bdr); border-radius:999px; padding:6px 10px;
    box-shadow:0 1px 2px rgba(0,0,0,.04);
  }}

  /* ipywidgets polish to match earlier cells */
  .mmc4-app .widget-checkbox input {{ accent-color: var(--acc) !important; }}
  .mmc4-app .widget-button {{
    border-radius: 12px !important;
    border: 0 !important;
    background: var(--acc) !important;
    color: #fff !important;
    box-shadow: 0 6px 18px rgba(116,198,157,.35);
    font-weight: 700;
  }}
  .mmc4-app .widget-dropdown select {{
    border:1px solid var(--card-bdr) !important;
    background:#fff !important;
    color:#0f172a !important;
    border-radius:12px !important;
    padding:6px 8px !important;
    box-shadow:0 1px 2px rgba(16,24,40,.06) inset;
  }}
  .mmc4-app .widget-selectmultiple select {{
    border:1px solid var(--card-bdr) !important;
    background:#fff !important;
    border-radius:12px !important;
    padding:6px 8px !important;
    box-shadow:0 1px 2px rgba(16,24,40,.06) inset;
  }}
  .mmc4-app .widget-floattext input {{
    border:1px solid var(--card-bdr) !important;
    border-radius:12px !important;
    background:#fff !important;
    box-shadow:0 1px 2px rgba(16,24,40,.06) inset;
  }}
  .mmc4-app .widget-floatslider .noUi-connect,
  .mmc4-app .widget-floatslider .noUi-handle {{
    outline:none !important;
  }}

  /* KPI card */
  .mmc4-kpi {{
    border-radius: 14px;
    padding: 14px 16px;
    border: 1px solid var(--card-bdr);
    background: #fff;
    min-width: 190px;
    box-shadow: 0 2px 10px rgba(0,0,0,0.04);
  }}
  .mmc4-kpi .t {{ font-size:11px; text-transform:uppercase; letter-spacing:.08em; color:#6b7b76; margin-bottom:8px; }}
  .mmc4-kpi .v {{ font-size:22px; font-weight:800; color:var(--heading); }}
  .mmc4-kpi .s {{ font-size:12px; color:#6b767f; }}

  /* Transparent pandas Styler */
  .mmc4-app table.dataframe,
  .mmc4-app table.dataframe thead tr,
  .mmc4-app table.dataframe tbody tr,
  .mmc4-app table.dataframe th,
  .mmc4-app table.dataframe td {{ background: transparent !important; }}

</style>
"""))

# ------------------------ Configuration ------------------------
csb_XLSM_PATH = Path(os.getenv("XLSM_PATH", "MMC_LCA_Consolidated_Spreadsheet_MacroEnabled.xlsm"))

# ------------------------ Helpers: fuzzy matching + sheet/header detection ------------------------
def csb_canon(s: str) -> str:
    return re.sub(r"[^a-z0-9]+", "", str(s or "").strip().lower())

def csb_row_header_score(row_cells) -> float:
    cols = [str(c or "").strip() for c in row_cells]
    n = len(cols) or 1
    non_empty  = sum(bool(c) and not c.lower().startswith("unnamed") for c in cols)
    unique     = len(set(cols))
    lettersdom = sum((sum(ch.isalpha() for ch in c) >= sum(ch.isdigit() for ch in c)) for c in cols)
    return (3.0 * (non_empty/n) + 1.5 * (unique/n) + 0.7 * (lettersdom/n))

def csb_alias_hits(row_cells, alias_bag) -> int:
    cset = {csb_canon(c) for c in row_cells if c is not None}
    hits = 0
    for alias_list in alias_bag.values():
        for a in alias_list:
            ca = csb_canon(a)
            if any(ca in col or col in ca for col in cset):
                hits += 1
                break
    return hits

def csb_best_sheet_name(xls: pd.ExcelFile, targets) -> str:
    for t in targets:
        tgt = csb_canon(t)
        for name in xls.sheet_names:
            if csb_canon(name) == tgt or tgt in csb_canon(name):
                return name
    return xls.sheet_names[0]

def csb_find_header_index(path: Path, sheet_name: str, alias_bag, search_rows=30) -> int:
    raw = pd.read_excel(path, sheet_name=sheet_name, header=None, nrows=search_rows, engine="openpyxl")
    best_idx, best_tup = 0, (-1e9, -1e9)
    for i in range(min(search_rows, len(raw))):
        row = list(raw.iloc[i].values)
        cand = (csb_alias_hits(row, alias_bag), csb_row_header_score(row))
        if cand > best_tup: best_tup, best_idx = cand, i
    return int(best_idx)

def csb_pick_col(df: pd.DataFrame, aliases) -> str | None:
    canon_map = {csb_canon(c): c for c in df.columns.astype(str)}
    for a in aliases:
        ca = csb_canon(a)
        if ca in canon_map: return canon_map[ca]
    for a in aliases:
        ca = csb_canon(a)
        for c in df.columns.astype(str):
            cc = csb_canon(c)
            if ca in cc or cc in ca: return c
    return None

# ------------------------ Column aliases for CSB table ------------------------
csb_REQUIRED = {
    "mmc_method": ["mmc_method","mmc method","mmc","mmc category","system type","method","method name","mmc_type"],
    "method_subtype": ["method_subtype","method subtype","system subtype","subtype"],
    "complete_system_name_concatenated": [
        "complete_system_name_concatenated","complete system name concatenated",
        "complete system name","complete system","system name concatenated","complete name"
    ],
    "system_name_short": ["system_name_short","system short","system name short","system short name","sys short"],
    "base_system": ["base_system","base system"],
    "ext_finish_name_short": ["ext_finish_name_short","ext finish name short","external finish name short","ext_finish","external finish","ext finish"],
    "int_finish_name_short": ["int_finish_name_short","int finish name short","internal finish name short","int_finish","internal finish","int finish"],
    "total_gwp": ["total_gwp","gwp total","total gwp","gwp (kgco2e/m2)","gwp kgco2e m2"],
    "total_r_value": ["total_r_value","r total","r value total","r_total (m2 k / w)","r_total"],
    "total_u_value": ["total_u_value","u total","u value total","u_total (w / m2 k)","u_total"],
    "total_mass": ["total_mass","mass total","total mass","mass (kg/m2)","kg/m2"],
}
csb_OPTIONAL_NUM = {
    "base_gwp": ["base_gwp","base gwp"],
    "base_r_value": ["base_r_value","base r value"],
    "base_mass": ["base_mass","base mass"],
    "ext_finish_gwp": ["ext_finish_gwp","ext gwp","external finish gwp"],
    "ext_finish_r_value": ["ext_finish_r_value","ext r value","external finish r value"],
    "ext_finish_mass": ["ext_finish_mass","ext mass","external finish mass"],
    "int_finish_gwp": ["int_finish_gwp","int gwp","internal finish gwp"],
    "int_finish_r_value": ["int_finish_r_value","int r value","internal finish r value"],
    "int_finish_mass": ["int_finish_mass","int mass","internal finish mass"],
}
csb_HEADER_ALIAS_BAG = {k: (csb_REQUIRED[k] if k in csb_REQUIRED else v) for k, v in {**csb_REQUIRED}.items()}

# ------------------------ XLSM readers ------------------------
def csb_read_complete_system_df_from_xlsm(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f"[CSB] XLSM file not found: {path}")
    xls = pd.ExcelFile(path)
    sheet = csb_best_sheet_name(xls, ["complete system builder","complete system master","complete systems","system builder"])
    hdr = csb_find_header_index(path, sheet, csb_HEADER_ALIAS_BAG, search_rows=30)
    print(f"[CSB] Using sheet '{sheet}' (header row index {hdr})")
    df = pd.read_excel(path, sheet_name=sheet, header=hdr, engine="openpyxl")
    df = df.loc[:, ~df.columns.astype(str).str.match(r"^\s*Unnamed:\s*\d+\s*$", na=False)]
    df = df.dropna(thresh=max(1, int(len(df.columns)*0.5)))
    return df

def csb_read_base_systems_df_from_xlsm(path: Path) -> pd.DataFrame | None:
    if not path.exists(): return None
    try: xls = pd.ExcelFile(path)
    except Exception: return None
    sheet = csb_best_sheet_name(xls, ["base systems","base system","base system descriptions","base system info"])
    try:
        tmp = pd.read_excel(path, sheet_name=sheet, header=None, nrows=25, engine="openpyxl")
        best_idx, best_score = 0, -1e9
        for i in range(min(25, len(tmp))):
            score = csb_row_header_score(list(tmp.iloc[i].values))
            if score > best_score: best_idx, best_score = i, score
        df = pd.read_excel(path, sheet_name=sheet, header=best_idx, engine="openpyxl")
        df = df.loc[:, ~df.columns.astype(str).str.match(r"^\s*Unnamed:\s*\d+\s*$", na=False)]
        df = df.dropna(thresh=max(1, int(len(df.columns)*0.4)))
        return df
    except Exception:
        return None

# ------------------------ Load & normalize ------------------------
def csb_try_xlsm() -> tuple[pd.DataFrame, pd.DataFrame | None, str]:
    df = csb_read_complete_system_df_from_xlsm(csb_XLSM_PATH)
    base_df = csb_read_base_systems_df_from_xlsm(csb_XLSM_PATH)
    return df, base_df, f"XLSM:{csb_XLSM_PATH.name}"

csb_df_raw, csb_base_raw, csb_source = csb_try_xlsm()
print(f"[CSB] Loaded Complete System data from: {csb_source}")

def csb_map_columns(df: pd.DataFrame) -> pd.DataFrame:
    colmap = {}
    for logical, aliases in csb_REQUIRED.items():
        found = csb_pick_col(df, aliases)
        if found is None:
            raise RuntimeError(f"[CSB] Required column not found: {logical} (aliases: {aliases})")
        colmap[logical] = found
    for logical, aliases in csb_OPTIONAL_NUM.items():
        found = csb_pick_col(df, aliases)
        if found is not None: colmap[logical] = found
    out = df[list(colmap.values())].copy()
    out.columns = list(colmap.keys())
    numeric_cols = [c for c in ["total_gwp","total_r_value","total_u_value","total_mass"] if c in out.columns]
    numeric_cols += [c for c in csb_OPTIONAL_NUM.keys() if c in out.columns]
    for c in numeric_cols: out[c] = pd.to_numeric(out[c], errors="coerce")
    for c in ["mmc_method","method_subtype","complete_system_name_concatenated",
              "system_name_short","base_system","ext_finish_name_short","int_finish_name_short"]:
        if c in out.columns: out[c] = out[c].astype(str).where(out[c].notna(), "")
    return out

csb_env = csb_map_columns(csb_df_raw)

# ------------------------ Base systems lookup ------------------------
def csb_build_base_lookup(df: pd.DataFrame | None):
    if df is None or df.empty: return {}
    key_col = None
    for c in df.columns:
        cl = c.lower()
        if 'base' in cl and 'system' in cl: key_col = c; break
    desc_col = None; comps_col = None
    for c in df.columns:
        cl = c.lower()
        if desc_col is None and ('desc' in cl or 'overview' in cl or 'summary' in cl): desc_col = c
        if comps_col is None and ('component' in cl or 'included' in cl or 'bill' in cl): comps_col = c
    lut = {}
    for _, r in df.iterrows():
        k = str(r.get(key_col, '')).strip()
        if not k: continue
        lut[k] = {'desc': str(r.get(desc_col, '') or '').strip(), 'comps': str(r.get(comps_col, '') or '').strip()}
    return lut

_CSB_BASE_LUT = csb_build_base_lookup(csb_base_raw)
def csb_lookup_base_info(base_key: str): return _CSB_BASE_LUT.get(str(base_key).strip(), {})

# ------------------------ Friendly finish bucketing ------------------------
def csb_friendly_ext_from_short(s):
    k = re.sub(r'[^a-z0-9]+','', str(s).lower())
    if 'com' in k and ('fiber' in k or 'fibre' in k): return "Composite Fiber"
    if 'epd' in k and 'brick' in k: return "Solid Brick (Type EPD)"
    if 'lca' in k and 'brick' in k: return "Solid Brick (Type LCA)"
    if 'brick' in k: return "Solid Brick"
    return s or "—"

def csb_friendly_int_from_short(s):
    k = str(s).lower()
    if 'gyp' in k or 'gplas' in k or 'gypsum' in k: return "Gypsum Plasterboard"
    return s or "—"

csb_env['ext_friendly'] = csb_env['ext_finish_name_short'].map(csb_friendly_ext_from_short)

# ------------------------ Viability integration ------------------------
def _csb_norm_key(s): return str(s).strip().casefold().replace(" ","").replace("-","").replace("_","")
csb_full = csb_env.copy()
csb_active = csb_full.copy()

def csb_apply_viability_list(viable_list):
    global csb_active
    keyset = {_csb_norm_key(x) for x in (viable_list or [])}
    if keyset:
        mask = csb_full['mmc_method'].astype(str).map(_csb_norm_key).isin(keyset)
        csb_active = csb_full.loc[mask].copy()
    else:
        csb_active = csb_full.copy()

if 'VIABLE_METHODS_SET' in globals() and globals()['VIABLE_METHODS_SET']:
    csb_apply_viability_list(globals()['VIABLE_METHODS_SET'])
elif 'ACTIVE_VIABLE_METHODS' in globals() and globals()['ACTIVE_VIABLE_METHODS']:
    csb_apply_viability_list(globals()['ACTIVE_VIABLE_METHODS'])

print(f"[CSB] Rows after cleaning/filtering: {len(csb_active)}")

def apply_viability_to_complete_selector(results_df):
    viable = (results_df.loc[results_df['Viable'] == True, 'MMC_Method']
              .dropna().astype(str).tolist())
    globals()['ACTIVE_VIABLE_METHODS'] = viable
    csb_apply_viability_list(viable)
    csb_do_refresh()

# ------------------------ UI widgets ------------------------
mmc_dd      = w.Dropdown(description="", layout=w.Layout(width='240px'))
subtype_dd  = w.Dropdown(description="", layout=w.Layout(width='240px'))
ext_dd      = w.Dropdown(description="", layout=w.Layout(width='240px'))
int_dd      = w.Dropdown(description="", options=['Gypsum Plasterboard'], layout=w.Layout(width='240px'))
systems_ms  = w.SelectMultiple(description="", rows=7, layout=w.Layout(width='100%'))

select_all_systems = w.Checkbox(description='Select all under current filters', value=True, indent=False)
csb_L_in = w.FloatText(description="", value=2.4, step=0.1, layout=w.Layout(width='160px'))
csb_H_in = w.FloatText(description="", value=2.4, step=0.1, layout=w.Layout(width='160px'))

csb_compute_btn = w.Button(description="Compute", button_style="", icon="check")
csb_refresh_btn = w.Button(description="Refresh", button_style="", icon="refresh")

status_chip = w.HTML()

csb_cards_out    = w.Output()
csb_compare_out  = w.Output()
csb_msg_out      = w.Output()
csb_viz_heat_out = w.Output()
csb_viz_rank_out = w.Output()

# --- Weight controls (in a tidy block) ---
csb_w_R = w.FloatSlider(description='R (↑)',   value=0.25, min=0.0, max=1.0, step=0.05,
                        readout_format='.2f', continuous_update=False)
csb_w_U = w.FloatSlider(description='U (↓)',   value=0.25, min=0.0, max=1.0, step=0.05,
                        readout_format='.2f', continuous_update=False)
csb_w_G = w.FloatSlider(description='GWP (↓)', value=0.25, min=0.0, max=1.0, step=0.05,
                        readout_format='.2f', continuous_update=False)
csb_w_M = w.FloatSlider(description='Mass (↓)',value=0.25, min=0.0, max=1.0, step=0.05,
                        readout_format='.2f', continuous_update=False)
csb_w_reset = w.Button(description='Reset weights', icon='undo')

# ------------------------ Refresh logic ------------------------
def csb_do_refresh(_=None):
    with csb_msg_out: clear_output()
    with csb_cards_out: clear_output()
    with csb_compare_out: clear_output()
    with csb_viz_heat_out: clear_output()
    with csb_viz_rank_out: clear_output()

    mmc_opts = sorted(csb_active['mmc_method'].dropna().astype(str).unique().tolist())
    mmc_dd.options = [''] + mmc_opts
    mmc_dd.value = (mmc_opts[0] if len(mmc_opts) == 1 else '')

    ext_dd.options = [''] + ["Composite Fiber", "Solid Brick (Type EPD)", "Solid Brick (Type LCA)"]
    ext_dd.value = ''

    int_dd.options = ['Gypsum Plasterboard']
    int_dd.value = 'Gypsum Plasterboard'

    subtype_dd.options = ['']; subtype_dd.value = ''
    systems_ms.options = []; systems_ms.value = ()
    csb_refresh_subtypes()

    status_chip.value = ""  # cleared until compute

def csb_slice_active():
    q = csb_active.copy()
    if mmc_dd.value:
        q = q[q['mmc_method'].astype(str) == mmc_dd.value]
    if subtype_dd.value:
        q = q[q['method_subtype'].astype(str) == subtype_dd.value]
    if ext_dd.value:
        q = q[q['ext_friendly'].astype(str) == ext_dd.value]
    return q

def csb_refresh_subtypes(*_):
    with csb_msg_out: clear_output()
    sub = csb_active.copy()
    if mmc_dd.value:
        sub = sub[sub['mmc_method'].astype(str) == mmc_dd.value]
    opts = sorted(sub['method_subtype'].dropna().astype(str).unique().tolist())
    if not opts:
        subtype_dd.options = ['']; subtype_dd.value = ''
        csb_refresh_systems(); return
    subtype_dd.options = [''] + opts
    subtype_dd.value = (opts[0] if len(opts) == 1 else '')
    csb_refresh_systems()

def csb_refresh_systems(*_):
    with csb_msg_out: clear_output()
    q = csb_slice_active()
    names = sorted(q['complete_system_name_concatenated'].dropna().astype(str).unique().tolist())
    systems_ms.options = names
    systems_ms.value = tuple(names) if select_all_systems.value else tuple(names[:min(3, len(names))])

def csb_on_select_all_change(change):
    if change['new']:
        systems_ms.value = tuple(systems_ms.options)
    else:
        systems_ms.value = tuple(systems_ms.options[:min(3, len(systems_ms.options))])
select_all_systems.observe(csb_on_select_all_change, names='value')

# ------------------------ Normalization util ------------------------
def _csb_norm01(series, invert=False):
    s = pd.to_numeric(series, errors='coerce')
    if s.nunique(dropna=True) <= 1:
        z = pd.Series(0.5, index=s.index)
    else:
        mn, mx = s.min(), s.max()
        z = (s - mn) / (mx - mn)
    z = (1.0 - z) if invert else z
    return z.fillna(0.5)

# ------------------------ Viz 1: Heatmap (transparent + frameless) ------------------------
def csb_render_heatmap(cmp_df: pd.DataFrame, title="Performance heatmap (higher = better)"):
    df = cmp_df.reset_index(drop=True).copy()
    Z = np.vstack([
        _csb_norm01(df["R total (m²·K/W)"], invert=False),
        _csb_norm01(df["U total (W/m²·K)"], invert=True),
        _csb_norm01(df["GWP (kgCO₂e/m²)"],  invert=True),
        _csb_norm01(df["Mass (kg/m²)"],     invert=True),
    ]).T

    labels_cols = ["R (↑)", "U (↓)", "GWP/m² (↓)", "Mass/m² (↓)"]
    y_labels = df["System"].astype(str).tolist()
    n = len(y_labels)

    pastel_cmap = LinearSegmentedColormap.from_list(
        "pastel_better",
        ["#f8fafc", "#e8f6ef", "#d7f2e1", "#c6edd2", "#b3e6c3", "#a3deb6", "#8bd3a8"]
    )

    fig_h = max(2.6, 0.5 * n + 1.6)
    fig, ax = plt.subplots(figsize=(10.5, fig_h))
    fig.patch.set_alpha(0); ax.set_facecolor('none')
    for sp in ax.spines.values(): sp.set_visible(False)
    ax.tick_params(colors="#374151")

    im = ax.imshow(Z, aspect="auto", cmap=pastel_cmap, vmin=0, vmax=1, origin="upper")
    ax.set_xticks(range(len(labels_cols))); ax.set_xticklabels(labels_cols)
    ax.set_yticks(range(n)); ax.set_yticklabels(y_labels)
    ax.set_title(title, fontsize=14, fontweight="bold", color=HEADING)

    for r in range(n + 1): ax.axhline(r - 0.5, color="#ffffff", linewidth=0.8, alpha=0.9)
    for c in range(len(labels_cols) + 1): ax.axvline(c - 0.5, color="#ffffff", linewidth=0.8, alpha=0.9)

    if n <= 20:
        for i in range(n):
            for j in range(4):
                ax.text(j, i, f"{Z[i, j]*100:.0f}", va="center", ha="center", fontsize=8, color="#334155")

    cbar = fig.colorbar(im, ax=ax, fraction=0.035, pad=0.02)
    cbar.set_label("Normalized performance (0–1)", rotation=90, color="#374151")
    cbar.ax.tick_params(colors="#374151"); cbar.outline.set_visible(False); cbar.ax.set_facecolor('none')
    plt.tight_layout(); plt.show()

# ------------------------ Viz 2: Weighted ranking (transparent + frameless) ------------------------
def _csb_weights():
    raw = np.array([csb_w_R.value, csb_w_U.value, csb_w_G.value, csb_w_M.value], dtype=float)
    if not np.isfinite(raw).all() or raw.sum() <= 0:
        return np.array([0.25, 0.25, 0.25, 0.25])
    return raw / raw.sum()

def csb_render_weighted_ranking(cmp_df: pd.DataFrame,
                                title="Multi-criteria ranking (stacked; higher = better)"):
    if cmp_df.empty: return
    df = cmp_df.reset_index(drop=True).copy()

    N_R = _csb_norm01(df["R total (m²·K/W)"], invert=False)
    N_U = _csb_norm01(df["U total (W/m²·K)"],  invert=True)
    N_G = _csb_norm01(df["GWP (kgCO₂e/m²)"],   invert=True)
    N_M = _csb_norm01(df["Mass (kg/m²)"],      invert=True)

    W = _csb_weights()
    contrib = np.vstack([N_R*W[0], N_U*W[1], N_G*W[2], N_M*W[3]]).T
    score = contrib.sum(axis=1)

    plot_df = pd.DataFrame({
        "System": df["System"].astype(str),
        "Score": score,
        "R_contrib": contrib[:,0],
        "U_contrib": contrib[:,1],
        "G_contrib": contrib[:,2],
        "M_contrib": contrib[:,3],
    }).sort_values("Score", ascending=False).reset_index(drop=True)

    colors = {"R_contrib": "#ccebc5", "U_contrib": "#b3cde3", "G_contrib": "#a8ddb5", "M_contrib": "#fbb4ae"}
    labels = {"R_contrib": "R (↑)", "U_contrib": "U (↓)", "G_contrib": "GWP (↓)", "M_contrib": "Mass (↓)"}

    y = np.arange(len(plot_df))
    fig_h = max(2.8, 0.55*len(plot_df) + 1.8)
    fig, ax = plt.subplots(figsize=(10.5, fig_h))
    fig.patch.set_alpha(0); ax.set_facecolor('none')
    for sp in ax.spines.values(): sp.set_visible(False)
    ax.tick_params(colors="#374151")

    left = np.zeros(len(plot_df))
    for key in ["R_contrib","U_contrib","G_contrib","M_contrib"]:
        ax.barh(y, plot_df[key], left=left, color=colors[key], edgecolor="white", linewidth=0.8, label=labels[key])
        left += plot_df[key].values

    ax.set_yticks(y); ax.set_yticklabels(plot_df["System"])
    ax.invert_yaxis()
    ax.set_xlabel("Weighted score (0–1)", color="#374151")
    ax.set_title(title, fontsize=14, fontweight="bold", color=HEADING)
    ax.grid(axis="x", linestyle=":", linewidth=0.8, alpha=0.35)

    for yi, s in zip(y, plot_df["Score"]):
        ax.text(s + 0.01, yi, f"{s*100:.0f}", va="center", fontsize=9, color="#334155")

    ax.legend(loc="lower right", frameon=False, ncol=2)
    plt.tight_layout(); plt.show()

    Wtxt = f"Weights → R:{W[0]:.2f}, U:{W[1]:.2f}, GWP:{W[2]:.2f}, Mass:{W[3]:.2f} (normalized)"
    ipy_display(HTML(f"<div class='mmc4-subtle'>{Wtxt}</div>"))

# ------------------------ Compute & cards ------------------------
def _fmt(x, nd=3):
    try:
        if x is None: return "—"
        if isinstance(x, float) and (np.isnan(x) or np.isinf(x)): return "—"
        return f"{float(x):.{nd}f}"
    except Exception:
        return str(x)

def kpi(title, value, sub):
    return f'<div class="mmc4-kpi"><div class="t">{title}</div><div class="v">{_fmt(value)}</div><div class="s">{sub}</div></div>'

def csb_do_compute(_=None):
    with csb_msg_out: clear_output()
    with csb_cards_out: clear_output()
    with csb_compare_out: clear_output()
    with csb_viz_heat_out: clear_output()
    with csb_viz_rank_out: clear_output()

    if (not systems_ms.value) and systems_ms.options:
        systems_ms.value = tuple(systems_ms.options) if select_all_systems.value else tuple(systems_ms.options[:min(3, len(systems_ms.options))])

    if not systems_ms.value:
        with csb_msg_out:
            ipy_display(HTML("<div style='color:#b42318'>Select at least one Complete System Name.</div>"))
        return

    q = csb_slice_active()
    base = q[q['complete_system_name_concatenated'].astype(str).isin(list(systems_ms.value))][[
        'complete_system_name_concatenated','system_name_short','base_system',
        'ext_finish_name_short','int_finish_name_short',
        'total_gwp','total_r_value','total_u_value','total_mass'
    ]].drop_duplicates()

    if base.empty:
        with csb_msg_out:
            ipy_display(HTML("<div style='color:#b42318'>No matching rows for the selected names.</div>"))
        return

    area = float(csb_L_in.value) * float(csb_H_in.value)

    rows, blocks = [], []
    for _, r in base.iterrows():
        concat_name = str(r['complete_system_name_concatenated'])
        sys_short   = str(r['system_name_short']) if pd.notna(r['system_name_short']) else concat_name
        base_key    = str(r['base_system']) if pd.notna(r['base_system']) else ""
        ext_short   = str(r['ext_finish_name_short']) if pd.notna(r['ext_finish_name_short']) else ""
        int_short   = str(r['int_finish_name_short']) if pd.notna(r['int_finish_name_short']) else ""

        gwp_m2  = float(r['total_gwp'])
        R_val   = float(r['total_r_value'])
        U_val   = float(r['total_u_value'])
        mass_m2 = float(r['total_mass'])

        info = csb_lookup_base_info(base_key)
        desc_html  = "<div class='mmc4-subtle'></div>"
        comps_html = ""
        if info:
            base_desc = info.get("desc", "")
            comps     = info.get("comps", "")
            if isinstance(base_desc, str) and base_desc.strip():
                desc_html = f"<p><b>Base system overview:</b> {base_desc.strip()}</p>"
            if isinstance(comps, str) and comps.strip():
                items = "".join(f"<li>{x.strip()}</li>" for x in comps.split(",") if x.strip())
                if items: comps_html = f"<p><b>Base system — components included:</b></p><ul>{items}</ul>"

        formed   = f"{sys_short} + {ext_short or '—'} + {int_short or '—'}"
        ext_note = csb_friendly_ext_from_short(ext_short)
        int_note = csb_friendly_int_from_short(int_short)

        rows.append({
            "System": concat_name,
            "GWP total (kgCO₂e)": gwp_m2 * area,
            "R total (m²·K/W)":   R_val,
            "U total (W/m²·K)":   U_val,
            "Mass total (kg)":    mass_m2 * area,
            "GWP (kgCO₂e/m²)":    gwp_m2,
            "Mass (kg/m²)":       mass_m2
        })

        blocks.append(f"""
        <h3 style='margin:8px 0 4px;color:{HEADING}'>{concat_name}</h3>
        <div class="mmc4-subtle" style="margin:0 0 6px 0"><b>Formed as:</b> {formed}</div>
        <div class='desc-block' style='margin-top:8px'>
          {desc_html}
          {comps_html}
          <p><b>Exterior finish:</b> {ext_note}</p>
          <p><b>Interior finish:</b> {int_note}</p>
        </div>
        <div style="display:flex;gap:12px;flex-wrap:wrap;align-items:stretch;">
          {kpi("Total GWP", gwp_m2*area, "kgCO₂e")}
          {kpi("Total R Value", R_val, "m²·K/W")}
          {kpi("Total U Value", U_val, "W/m²·K")}
          {kpi("Total Mass", mass_m2*area, "kg")}
        </div>
        <div class="mmc4-subtle">Area: {_fmt(area)} m² · Reference: {_fmt(gwp_m2)} kgCO₂e/m², {_fmt(mass_m2)} kg/m²</div>
        """)

    with csb_cards_out:
        parts = "<div class='mmc4-label' style='font-size:13px'>Project Impact</div>" + \
                "<hr class='mmc4-hr'>".join(blocks)
        ipy_display(HTML(parts))

    cmp_df = pd.DataFrame(rows)[[
        "System","GWP total (kgCO₂e)","R total (m²·K/W)","U total (W/m²·K)","Mass total (kg)",
        "GWP (kgCO₂e/m²)","Mass (kg/m²)"
    ]].sort_values(by="GWP (kgCO₂e/m²)", ascending=True, na_position="last").reset_index(drop=True)

    with csb_compare_out:
        ipy_display(HTML("<div class='mmc4-label' style='font-size:13px'>Comparison (sorted by GWP per m²)</div>"))
        try:
            sty = (cmp_df.style
                   .hide(axis="index")
                   .set_table_attributes('class="dataframe" style="border-collapse:collapse; background:transparent; width:100%;"'))
            ipy_display(sty)
        except Exception:
            ipy_display(cmp_df)

    # Visuals
    global csb_last_cmp_df
    csb_last_cmp_df = cmp_df

    with csb_viz_heat_out:
        clear_output()
        csb_render_heatmap(csb_last_cmp_df)

    with csb_viz_rank_out:
        clear_output()
        csb_render_weighted_ranking(csb_last_cmp_df)

    # Update status chip
    n_sys = len(list(systems_ms.value))
    status_chip.value = (
        f"<span class='mmc4-chip'>Systems: <b>{n_sys}</b></span>&nbsp;"
        f"<span class='mmc4-chip'>Area: <b>{_fmt(area,2)}</b> m²</span>"
    )

# ------------------------ Weight handlers (live update) ------------------------
def _csb_on_weight_change(_):
    if 'csb_last_cmp_df' in globals() and csb_last_cmp_df is not None and len(csb_last_cmp_df) > 0:
        with csb_viz_rank_out:
            clear_output()
            csb_render_weighted_ranking(csb_last_cmp_df)

for _s in (csb_w_R, csb_w_U, csb_w_G, csb_w_M):
    _s.observe(_csb_on_weight_change, names='value')

def _csb_reset_weights(_):
    csb_w_R.value, csb_w_U.value, csb_w_G.value, csb_w_M.value = 0.20, 0.30, 0.40, 0.10
csb_w_reset.on_click(_csb_reset_weights)

# ------------------------ Wire up & render ------------------------
csb_refresh_btn.on_click(csb_do_refresh)
mmc_dd.observe(csb_refresh_subtypes, 'value')
subtype_dd.observe(csb_refresh_systems, 'value')
ext_dd.observe(csb_refresh_systems, 'value')
csb_compute_btn.on_click(csb_do_compute)

# ---- Labels above inputs (consistent with other sections) ----
lbl_mmc   = w.HTML("<div class='mmc4-label'>MMC Method</div>")
lbl_sub   = w.HTML("<div class='mmc4-label'>Method Subtype</div>")
lbl_ext   = w.HTML("<div class='mmc4-label'>External Finish</div>")
lbl_int   = w.HTML("<div class='mmc4-label'>Internal Finish</div>")
lbl_len   = w.HTML("<div class='mmc4-label'>Length (m)</div>")
lbl_hei   = w.HTML("<div class='mmc4-label'>Height (m)</div>")
lbl_sys   = w.HTML("<div class='mmc4-label'>System(s)</div>")

inputs_row = w.GridBox(
    children=[
        w.VBox([lbl_mmc, mmc_dd]),
        w.VBox([lbl_sub, subtype_dd]),
        w.VBox([lbl_ext, ext_dd]),
        w.VBox([lbl_int, int_dd]),
    ],
    layout=w.Layout(grid_template_columns="repeat(4, minmax(220px, 1fr))", grid_gap="12px", width='100%')
)

dim_row = w.GridBox(
    children=[
        w.VBox([lbl_len, csb_L_in]),
        w.VBox([lbl_hei, csb_H_in]),
        w.VBox([w.HTML("<div class='mmc4-label'>&nbsp;</div>"),
                w.HBox([csb_compute_btn, csb_refresh_btn], layout=w.Layout(gap='8px'))]),
    ],
    layout=w.Layout(grid_template_columns="repeat(3, minmax(220px, 1fr))", grid_gap="12px", width='100%')
)

weight_block = w.VBox([
    w.HTML("<div class='mmc4-label' style='font-size:13px'>Adjust priority (weights auto-normalize)</div>"),
    w.HBox([csb_w_R, csb_w_U, csb_w_G, csb_w_M], layout=w.Layout(gap='12px', flex_flow='row wrap')),
    csb_w_reset
], layout=w.Layout(gap='8px'))

viz_tabs = w.Tab(children=[csb_viz_heat_out, w.VBox([weight_block, csb_viz_rank_out], layout=w.Layout(gap='8px'))])
viz_tabs.set_title(0, "Heatmap")
viz_tabs.set_title(1, "Ranking")

csb_card = w.VBox([
    w.HBox([
        w.VBox([
            w.HTML("<div class='mmc4-title'>Complete System Selector → Project Impact</div>"),
            w.HTML("<div class='mmc4-subtle'>Select systems, set project area, then compare with transparent, frameless charts.</div>")
        ], layout=w.Layout(flex='1 1 auto')),
        status_chip
    ], layout=w.Layout(align_items='center', justify_content='space-between')),
    w.HTML("<hr class='mmc4-hr'>"),
    inputs_row,
    w.HTML("<div class='mmc4-subtle' style='margin-top:4px'>Ext Finish options are grouped as: Composite Fiber · Solid Brick (Type EPD) · Solid Brick (Type LCA)</div>"),
    w.HTML("<hr class='mmc4-hr'>"),
    w.VBox([lbl_sys, select_all_systems, systems_ms]),
    w.HTML("<hr class='mmc4-hr'>"),
    dim_row,
    w.HTML("<hr class='mmc4-hr'>"),
    csb_cards_out,
    csb_compare_out,
    viz_tabs,
    csb_msg_out
], layout=w.Layout(width="100%"), _dom_classes=['mmc4-app'])

[CSB] Using sheet 'COMPLETE SYSTEM BUILDER' (header row index 2)
[CSB] Loaded Complete System data from: XLSM:MMC_LCA_Consolidated_Spreadsheet_MacroEnabled.xlsm
[CSB] Rows after cleaning/filtering: 75


### Complete System Selection UI

In [26]:
# Initial render
csb_do_refresh()
ipy_display(csb_card)

# ------------- Optional bridge: auto-refresh after Section 1 "Apply Viability" -------------
def refresh_csb_from_viability():
    if 'VIABLE_METHODS_SET' in globals() and VIABLE_METHODS_SET:
        csb_apply_viability_list(VIABLE_METHODS_SET)
    elif 'ACTIVE_VIABLE_METHODS' in globals() and ACTIVE_VIABLE_METHODS:
        csb_apply_viability_list(ACTIVE_VIABLE_METHODS)
    csb_do_refresh()

if 'apply_btn' in globals():
    apply_btn.on_click(lambda _: refresh_csb_from_viability())